# Introduction

I devote considerable effort in this post to a mathematical enunciation of LSTM. A thorough derivation of the LSTM network is generally skipped in finance deep learning literature that features the LSTM. I seek to understand each aspect of the operation of RNN and LSTM's elegant and effective systems in considerable depth. I  describe firstly the RNN, followed by its evolution to the LSTM form, much more completely than is usually the case in finance DL papers, showing in detail the key operations I build in the empirical study in order to predict a financial time series.  These include unrolling the RNN, the motivations for the LSTM, namely managing the phenomena of vanishing gradients, and the subsequent mechanics of learning, which are the forward and backward passes and gradient descent.
$\\[0.1in]$

Few papers in the literature appear to do this systematically and comprehensively, and the best treatment I have found is that of Sherstinsky (2018, 2020). As with much of the most useful empirical DL literature, Sherstinsky's papers are not finance papers and considerable work is required to transcribe the most relevant aspects of it to a finance setting. Sherstinsky (2020) uses a signal processing framework and notation from engineering and life sciences. He derives a canonical RNN set up and, within this framework, explains RNN unrolling. I show that a neural network at the simplest level of abstraction takes an input, $x_t$, log returns specifically in the case of this paper, and outputs a value, in practice represented in a normalized form. The loop (i.e. the 'recurrent' aspect) of the RNN allows information to be passed sequentially through steps of the neural network, from one step to the next, thus creating a structure for sequential learning.  I unroll a RNN, following the framework of Sherstinsky (2020), to reveal its chain-like nature and intimate relation to sequences. Viewed in this way, the RNN is an obvious and compelling candidate for time series analysis. I then proceed to describe fully the training routines, Forward Pass (FP), Back Propagation (BP) and Gradient Descent (GD), along with the numerical phenomenon of vanishing gradients, first described by Hochreiter in his PhD thesis (1991), and subsequently addressed in the seminal LSTM paper of Hochreiter & Schmidhuber (1997). Sherstinsky provides a particularly complete enunciation of FP, BP, and GD, which I follow.  However, his overall framework is not particularly helpful from a finance perspective and considerable work is required to unpick it so that it readily ties back to the generally far briefer presentations that typify finance DL papers.
$\\[0.1in]$

# LSTM

## LSTM in finance DL literature

Finance researchers approaching DL, and the LSTM model specifically, appear rarely, if ever, to exposit the model fully.  An obvious way to do this is to start with the RNN, as, for example, Verstyuk (2020) and Sezer, Gudelek & Ozbayoglu (2019) do, but they are exceptions. The RNN equation is generally expressed in broader (i.e. non-finance) DL literature via input, $x$, and weight, $w$, terms, using a form that looks something like this:

\begin{align}
    h_t &= W f \left( h_{t-1} \right) + W ^{(hx)} x_t \label{5.1} \tag{5.1} \\ \\
    y_t &= W^S f \left( h_t \right) \label{5.2} \tag{5.2}
\end{align}
$\\[0.1in]$

where $W$ denotes weight matrices and $h_t$ is an output value that in practice must be represented in a normalized form, $y_t$. Equation $\ref{5.1}$ thus describes an output of a single RNN neuron, or node, whilst equation $\ref{5.2}$ describes a nonlinear activation function that provides the cumulative output of all preceding neurons. The activation function thus transforms summed weighted input from the neuron into the activation of the neuron, or equivalently its output for that input.  
$\\[0.1in]$

But it is more common for finance DL  papers addressing the LSTM to skip its origins and relation to the canonical RNN and proceed directly to a description of a system of equations, usually comprising 5 equations, that are held out to fully define a LSTM neuron, or cell as it is more commonly called in the case of the LSTM RNN. 
$\\[0.1in]$

The LSTM specializes the general RNN by introducing state vectors and it permits an input sequence, for example a financial returns time-series, to condition upon recent temporal dynamics. Its input variable, $x_t$, is in fact a lagged prediction of the output, thus:

\begin{align}
    x_t := y_{t-1}
    \label{5.3} \tag{5.3}
\end{align}
$\\[0.1in]$

And the LSTM is generally represented as complete in a form resembling that of Verstyuk (2020) or Sezer, Gudelek & Ozbayoglu (2019):

\begin{align}
    f_t &= \mathrm{tanh} \left( b_f + W_f x_t + U_f h_{t-1} \right)  \label{5.4} \tag{5.4} \\ \\
    o_t &= \mathrm{tanh} \left( b_o + W_o x_t + U_o h_{t-1} \right)  \label{5.5} \tag{5.5} \\ \\
    i_t &= \mathrm{tanh} \left( b_i + W_i x_t + U_i h_{t-1} \right)  \label{5.6} \tag{5.6} \\ \\
    c_t &= f_t \ast c_{t-1} + i_t \ast \mathrm{tanh} \left( b_c + W_c x_t + U_c h_{t-1} \right)  \label{5.7} \tag{5.7} \\ \\
    h_t &= o_t \ast \mathrm{tanh} \left( c_t \right)  \label{5.8} \tag{5.8}
\end{align}
$\\[0.1in]$

where:
$\\[0.1in]$

$x_t$ denotes the LSTM unit, or cell's, input vector.
$\\[0.03in]$

$f_t$ denotes the activation vector of a 'forget' gate.
$\\[0.03in]$

$o_t$ denotes the activation vector of an 'output' gate.
$\\[0.03in]$

$i_t$ denotes the activation function of an 'input' gate.
$\\[0.03in]$

$c_t$ denotes a vector that represents the LSTM cell's state.
$\\[0.03in]$

$W, U$ denote weight matrices to be trained or learned.
$\\[0.03in]$

$b$ denotes bias vector parameters to be trained or learned.
$\\[0.1in]$

However, I find expository discourse of the LSTM RNN in finance DL studies (see, for example, Verstyuk (2020), Petnehazi & Gall (2019), Nguyen, Tran, Gunawan & Kohn (2019), Kim & Kim (2019), Bucci (2019), Borovkova & Tsiamas (2019), Almosova & Andresen (2019), Ahmed, Smola, Wang, Xing, Zaheer & Zheng (2018), Namin & Namin (2018), Fischer & Krauss (2017), Xiong, Nichols & Shen (2016)) to be unsatisfying, not least because in execution it bears very little relation to the code one writes and uses to confront a LSTM RNN to a returns series. Theory is hardly pseudo-code, but the gap between what is presented in the methodology and what actually needs to be done in the empirical study is just sometimes too big in finance DL literature and too much generally seems to have been skipped or omitted. 
$\\[0.1in]$

I therefore wish to enunciate the RNN and its evolution to the LSTM form much more completely, showing in detail the key operations I build in order to predict a financial time series, such as unrolling a RNN, the motivations for the LSTM, namely managing the phenomena of vanishing gradients, and the subsequent mechanics of learning, namely the forward and backward passes and gradient descent.  Few papers in the literature appear to do this systematically and comprehensively, and the best treatment I have found is that of Sherstinsky (2018, 2020). As with much of the most useful empirical DL literature, Sherstinsky's papers are not finance papers and considerable work is required to transcribe the most relevant aspects of it to a finance setting. Sherstinsky (2020) nevertheless provides the most thorough exposition of the necessary RNN and LSTM operations, using a signal processing framework. 
$\\[0.1in]$

Sherstinsky derives a canonical RNN set up using notation from engineering and life sciences and, within this framework, explains RNN unrolling. I was unable to find any reference to the process of unrolling a RNN in the finance-specific DL literature.  Generally papers move either directly from a stark presentation of the canonical RNN equation to an equally stark formulation of the LSTM system, as shown in equations $\ref{5.1}$ to $\ref{5.8}$, or, more often, directly to the latter, i.e. equations $\ref{5.4}$ to $\ref{5.8}$ alone.  This is unfortunate, not least because it misses an important reason why the RNN, and in turn the LSTM, may be of interest to researchers of financial time series and volatility forecasting. 
$\\[0.1in]$

As I show in figures 1 and 2, a neural network at the simplest level of abstraction takes an input, $x_t$, log returns specifically in the case of this paper, and outputs a value, $h_t$, which, as noted above, in practice is represented in a normalized form, $y_t$. The loop of the RNN simply allows information to be passed sequentially through steps of the neural network, from one step to the next, thus creating a structure for sequential learning.  By unrolling a RNN (figure 2), I reveal its chain-like nature and intimate relation to sequences. Viewed in this way, the RNN is an obvious and compelling candidate for time series analysis. Sherstinsky unrolls the RNN, then proceeds to describe fully the training routines, FP, BP and GD, along with the numerical phenomenon of vanishing gradients, first described by Schmidhuber's student, Hochreiter in his PhD thesis (1991), and subsequently addressed in the seminal LSTM paper of Hochreiter & Schmidhuber (1997). Sherstinsky provides a particularly complete enunciation of FP, BP, and GD.  However, his overall framework is not particularly helpful from a finance perspective and considerable work is required to unpick it so that it readily ties back to the far briefer presentations that typify finance DL papers.
$\\[0.1in]$

## RNN

My $\mathrm{WMP}_t$ series, defined as the natural log of the weighted mid-price of the AAPL, JPM securities or the EURUSD exchange rate, follows the construction in equations $\ref{4.1}$ and $\ref{4.2}$, and I define the return innovations $x_t \equiv \mathrm{WMP}_t - \mathrm{WMP}_{t-1}$. I may write $x \left( t \right)$ in the form of an ordinary differential equation thus: 

\begin{align}
    \frac{dh \left( t \right)} {dt} = g \left( t \right) + \epsilon
    \label{5.9} \tag{5.9}
\end{align}
$\\[0.1in]$

where $h_t$ is again an output value that in practice must be represented in a normalized form, $y_t$, following equations $\ref{5.1}$ and $\ref{5.1}$, $g \left( t \right)$ is a vector function of time, and $\epsilon$ is a constant bias, or prediction error, term.
$\\[0.1in]$

I follow, structurally, Sherstinsky (2020), but re-align many concepts and notation with broader finance DL literature (for example, to fit more closely with equations $\ref{5.1}$ to $\ref{5.8}$ and the approaches taken by Verstyuk (2020), Petnehazi & Gall (2019), Nguyen, Tran, Gunawan & Kohn (2019), Kim & Kim (2019), Bucci (2019), Borovkova & Tsiamas (2019), Almosova & Andresen (2019), Ahmed, Smola, Wang, Xing, Zaheer & Zheng (2018), Namin & Namin (2018), Fischer & Krauss (2017), Xiong, Nichols & Shen (2016), despite the considerable discrepancy in approach across these studies).  Sherstinsky invokes a canonical form for $g \left( t \right)$ whereby it is written in the following form:

\begin{align}
    g \left( t \right) = j \left( h \left( t \right), x \left( t \right) \right)
    \label{5.10} \tag{5.10}
\end{align}
$\\[0.1in]$

where $x \left( t \right)$ is an input vector. Combining $\ref{5.9}$ and $\ref{5.10}$ yields:

\begin{align}
    \frac{dh \left( t \right)} {dt} = j \left( h \left( t \right), x \left( t \right) \right) + \epsilon
    \label{5.11} \tag{5.11}
\end{align}
$\\[0.1in]$

Following equation $\ref{5.10}$, a special case of $g \left( t \right)$ employed in ML, may be written thus:

\begin{align}
    g \left( t \right) = a \left( t \right) + b \left( t \right) + c \left( t \right)
    \label{5.12} \tag{5.12}
\end{align}
$\\[0.1in]$

where $a \left( t \right)$, $b \left( t \right)$, and $c \left( t \right)$ are terms that collectively constitute what is generally denoted the 'additive model' (Hastie, Tibshirani & Friedman (2009) describe the generalized additive model in Chapter 9).  In ML and DL the additive model adds terms that determine rates of change of neuronal activities (Grossberg 1988, 2013).
$\\[0.1in]$

Sherstinsky defines a saturating additive model with the three constituent terms, $a_t$, $b_t$, and $c_t$, which I redefine as follows:

\begin{align*}
    a_t &= \sum^{K_h - 1}_{k = 0} a_k \left( h \left( t - \tau_k \right) \right) \label{5.13} \tag{5.13} \\ \\
    b_t &= \sum^{K_y - 1}_{k = 0} b_k \left( y \left( t - \tau_k \right) \right) \label{5.14} \tag{5.14} \\ \\
    y \left( t - \tau_y \right) &= J \left( h \left( t - \tau_r \right) \right) \label{5.15} \tag{5.15} \\ \\
    c_t &= \sum^{K_x - 1}_{k = 0} c_k \left( x \left( t - \tau_k \right) \right) \label{5.16} \tag{5.16}
\end{align*}
$\\[0.1in]$

where $y \left( t \right)$ is an output vector that represents $h \left( t \right)$ in a normalized form. In practice, $J \left( \cdot \right)$, may be obtained with the hyperbolic tangent, $\left( \mathrm{tanh} \right)$. Substituting equations $\ref{5.13}$, $\ref{5.14}$, $\ref{5.15}$, and $\ref{5.16}$, into equation $\ref{5.12}$, then inserting into equation $\ref{5.9}$, yields:

\begin{align*}
    \frac{d} {dt} h \left( t \right) &= \sum^{K_h - 1}_{k = 0} a_k \left( h \left( t - \tau_k \right) \right) 
    + \sum^{K_y - 1}_{k = 0} b_k \left( y \left( t - \tau_k \right) \right) \\
    &+ \sum^{K_x - 1}_{k = 0} c_k \left( x \left( t - \tau_k \right) \right) + \epsilon \label{5.17} \tag{5.17} \\ \\
    y \left( t - \tau_y \right) &= J \left( h \left( t - \tau_r \right) \right)  \label{5.18} \tag{5.18} 
\end{align*}
$\\[0.1in]$

Equation $\ref{5.17}$ above is a type of nonlinear ordinary delay differential equation (DDE). It has discrete delays and its derivative at any given time may be obtained in terms of the values of it at previous times. The first component of Equation $\ref{5.17}$, $\sum^{K_h - 1}_{k = 0} a_k \left( h \left( t - \tau_k \right) \right)$, is the output function. The second component, $\sum^{K_y - 1}_{k = 0} b_k \left( y \left( t - \tau_k \right) \right)$, is the normalized output function. And the third component, $\sum^{K_x - 1}_{k = 0} c_k \left( x \left( t - \tau_k \right) \right))$, is the input function.  The hyperbolic tangent, $\mathrm{tanh}$, is generally used in practice as the normalization function.  The right hand side of equation $\ref{5.17}$ consists of time delay terms that in turn comprise the system's memory. Collectively these terms enable the respective contributions of the output, normalized output, and input.  Time delay in NNs arises from the interaction between neurons (see for example Kyrychko & Hogan (2010)) and is thus an natural part of the system and its dynamics. 
$\\[0.1in]$

Each term on the right hand side of equation $\ref{5.17}$ has a qualitatively different impact. $a_k \left( h \left( t - \tau_k \right) \right)$ has a stability impact, $b_k \left( y \left( t - \tau_k \right) \right)$, the normalized output vector, captures much of the information that determines long-term behavior. Explicit non-zero delay time constants account for the delays inherent in neural interactions (see for example de Vries & Principe (1991)). $K_h$, $K_y$, and $K_x$ thus represent counts of the functions $a_k \left( h \left( t - \tau_k \right) \right)$, $b_k \left( y \left( t - \tau_k \right) \right)$, and $c_k \left( x \left( t - \tau_k \right) \right)$ respectively, along with the associated counts of $\tau_h \left( k \right )$, $\tau_y \left( k \right )$, and $\tau_x \left( k \right )$ respectively. 
$\\[0.1in]$

Let $a_k \left( h \left( t - \tau_k \right) \right)$, $b_k \left( y \left( t - \tau_k \right) \right)$, and $c_k \left( x \left( t - \tau_k \right) \right)$ be linear functions of $h$, $y$, and $x$ respectively. Then I may re-write equation $\ref{5.17}$ with linear (matrix-valued) coefficients, thus:

\begin{align*}
    \frac{d} {dt} h \left( t \right) &= \sum^{K_h - 1}_{k = 0} A_k \left( h \left( t - \tau_k \right) \right) 
    + \sum^{K_y - 1}_{k = 0} B_k \left( y \left( t - \tau_k \right) \right) \\
    &+ \sum^{K_x - 1}_{k = 0} C_k \left( x \left( t - \tau_k \right) \right) + \epsilon \label{5.19} \tag{5.19} 
\end{align*}
$\\[0.1in]$

I apply the following simplification to equation $\ref{5.19}$:

\begin{align*}
    K_h &= 1 \\
    \tau_{h, 0} &= 0 \\
    A_0 &= A \\
    K_y &= 1 \\
    \tau_{y, 0} &= \tau_0 \\
    B_0 &= B \\
    K_x &= 1 \\
    \tau_{x, 0} &= 0 \\
    C_0 &= C 
    \label{5.20} \tag{5.20}
\end{align*}
$\\[0.1in]$

In order to re-write equation $\ref{5.19}$ thus:

\begin{align}
    \frac{d} {dt} h \left( t \right) = A h \left( t \right) + B y \left( t - \tau_0\right) + C x \left( t \right) + \epsilon
    \label{5.21} \tag{5.21}
\end{align}
$\\[0.1in]$

Equation $\ref{5.19}$ and $\ref{5.21}$ are non-linear first order DDE. Sherstinsky (2020) uses numerical integration to evaluate these equations, applying the backward Euler discretization rule (following Butcher, 2003).
$\\[0.1in]$

I denote a sampling time step duration $\Delta T$ and index time, $n$. I then apply the backward Euler method in the manner of Sherstinsky (2020) to equation $\ref{5.21}$, yielding:

\begin{align*}
    t &= n \Delta T \label{5.22} \tag{5.22} \\ \\
    \frac{d} {dt} h \left( t \right) &\sim \frac {h \left( n \Delta T + \Delta T \right) - h \left( n \Delta T \right)} {\Delta T} \label{5.23} \tag{5.23} \\ \\
    A h \left( t \right) + B y \left( t - \tau_0 \right) + C x \left( t \right) + \epsilon &= A h \left( n \Delta T \right) + B y \left( n \Delta T  - \tau_0 \right) \\
    &+ C x \left ( n \Delta T \right) + \epsilon \label{5.24} \tag{5.24} \\ \\
    A h \left( t + \Delta T \right) + B y \left( t + \Delta T - \tau_0 \right) + C x \left( t + \Delta T \right) + \epsilon &= A h \left( n \Delta T + \Delta T \right) + B y \left( n \Delta T  + \Delta T - \tau_0 \right) \\ 
    &+ C x \left ( n \Delta T + \Delta T \right) + \epsilon \label{5.25} \tag{5.25} \\ \\
    \frac {h \left( n \Delta T + \Delta T \right) - h \left( n \Delta T \right)} {\Delta T} &\sim A h \left( n \Delta T + \Delta T \right) + B y \left( n \Delta T  + \Delta T - \tau_0 \right) \\
    &+ C x \left ( n \Delta T + \Delta T \right) + \epsilon \label{5.26} \tag{5.26}
\end{align*}
$\\[0.1in]$

The delay, $\tau_0$, may be set equal to a single time step, so that the output and normalized output is read into memory at each time step and used in the above equations for the next time step as an ongoing recursion, for the time step duration, $T$.
$\\[0.1in]$

Setting $\tau_0 = \Delta T$ in equation $\ref{5.26}$ thus yields:

\begin{align*}
    \frac {h \left( n \Delta T + \Delta T \right) - h \left( n \Delta T \right)} {\Delta T} &= A h \left( n \Delta T + \Delta T \right) + B y \left( n \Delta T \right) + C x \left ( n \Delta T + \Delta T \right) + \epsilon \label{5.27} \tag{5.27} \\ \\
    \frac {h \left( \left( n + 1 \right) \Delta T \right) - h \left( n \Delta T \right)} {\Delta T} &= A h \left( \left( n + 1 \right) \Delta T \right) + B y \left( n \Delta T \right) + C x \left( \left( n + 1 \right) \Delta T \right) + \epsilon \label{5.28} \tag{5.28} \\ \\
    h \left( \left( n + 1 \right) \Delta T \right) - h \left( n \Delta T \right) &= \Delta T \left( A h \left( \left( n + 1 \right) \Delta T \right) + B y \left( n \Delta T \right) + C x \left( \left( n + 1 \right) \Delta T \right) + \epsilon \right) \label{5.29} \tag{5.29} 
\end{align*}
$\\[0.1in]$

As a result of the discretization in equation $\ref{5.29}$, measures of time become integral multiples of $\Delta T$. Omitting $\Delta T$ renders the time axis dimensionless and transforms the vectors into sequences and thus equation $\ref{5.21}$ into a non-linear first order difference equation, which I write thus:

\begin{align*}
    h_{t+1} - h_t &= \Delta T \left( A h_{t+1} + B y_t + C x_{t+1} + \epsilon \right ) \label{5.30} \tag{5.30} \\ \\
    h_{t+1} &= h_t + \Delta T \left( A h_{t+1} + B y_t + C x_{t+1} + \epsilon \right ) \\ \\
    \left( I - \left( \Delta T \right) A \right) h_{t+1} &= h_t + \left( \left( \Delta T \right) B \right) y_t + \left( \left( \Delta T \right) C \right) x_{t+1} + \left( \Delta T \right) \epsilon \label{5.31} \tag{5.31} 
\end{align*}
$\\[0.1in]$

I define a weight matrix, $W_h$, thus:

\begin{align}
    W_h = {\left( I - \left( \Delta T \right) A \right)}^{-1} \label{5.32} \tag{5.32} 
\end{align}
$\\[0.1in]$

Multiply both sides of equation $\ref{5.31}$ by $W_h$, thus: 

\begin{align*}
    h_{t+1} = W_h h_t
    + \left( \left( \Delta T \right) W_h B \right) y_t 
    + \left( \left( \Delta T \right) W_h C \right) x_{t+1} 
    + \left( \left( \Delta T \right) W_h \epsilon \right)
\end{align*}
$\\[0.1in]$

And move forward 1 step, thus yielding: 

\begin{align*}
    h_t &= W_h h_{t-1} + \left( \left( \Delta T \right) W_h B \right) y_{t-1} + \left( \left( \Delta T \right) W_h C \right) x_t + \left( \left( \Delta T \right) W_h \epsilon \right) \\ \\
    y_t &= J \left( h_t \right) \label{5.33} \tag{5.33}
\end{align*}
$\\[0.1in]$

I define two additional weight matrices, $W_y$ and $W_x$, along with a bias vector, $b_h$, thus:

\begin{align*}
    W_y &= \left( \Delta T \right) W_h B \label{5.34} \tag{5.34} \\ \\
    W_x &= \left( \Delta T \right) W_h C \label{5.35} \tag{5.35} \\ \\
    b_h &= \left( \Delta T \right) W_h \epsilon \label{5.36} \tag{5.36}
\end{align*}
$\\[0.1in]$

Finally, I express the above system in the canonical RNN form, following Sherstinsky (2020), Kornblith, Norouzi, Lee & Hinton (2019), and Schmidhuber (2015, 2013):

\begin{align*}
    h_t &= W_h h_{t-1} + W_y y_{t-1} + W_x x_t + b_h \label{5.37} \tag{5.37} \\ \\
    y_t &= J \left( h_t \right) \label{5.38} \tag{5.38}
\end{align*}
$\\[0.1in]$

Figure 1 provides an illustration of the canonical RNN formulation (equation $\ref{5.37}$). 
$\\[0.1in]$

![Figure 1. Canonical RNN cell](fig_1.png)

Equation $\ref{5.37}$ is stable if the eigenvalues of $\hat W = W_h + W_y$ are within the complex-valued unit circle (i.e., the complex number set that has a magnitude of one; see Sherstinsky, 2018, 2020). $A$ and $B$ has a very many elements potentially that satisfy this requirement, so that $\Delta T = 1$ may be set, for tractability.  As a result, $h_{t-1}$ has negligible impact and may be ignored, reducing equation $\ref{5.37}$ to a standard RNN definition (recall, for example, equation $\ref{3.3}$):

\begin{align*}
    h_t &= W_y y_{t-1} + W_x x_{t} + b_h \label{5.39} \tag{5.39} \\ \\
    y_t &= J \left( h_t \right) \label{5.40} \tag{5.40}
\end{align*}
$\\[0.1in]$

By adapting the framework of Sherstinsky (2020) to complement and considerably extend finance DL literature, I have shown that the RNN, in canonical form (equation $\ref{5.37}$) or equation $\ref{5.39}$, essentially implements the backward Euler method for the ordinary DDE $\ref{5.21}$. This 'forward' direction of starting in the continuous time domain (differential equation) and ending in the discrete domain (difference equation) implies that the phenomenon modeled by the econometrician is analog, and that it is modeled for purposes of realization in the discrete domain. For example, the source signal might be AAPL's price on the NYSE, as recorded on the NYSE TAQ. The orginal signal thus contains all information regarding the price of AAPL. The samples generated by a hypothetical discretization, governed in this model by equation $\ref{5.21}$, could be captured as sparse samples , of the kind, for example, used in the empirical study (see sections 4 and 6). In this scenario, it is the sparsely sampled WMP that the RNN reproduces, not the actual price of AAPL. The key, if somewhat subtle, point in this scenario is that applying the RNN as a model implies that the price of AAPL is governed by equation $\ref{5.21}$, so that the role of the RNN is that of implementing the computational method for solving this delay differential equation using the backward Euler discretization rule, under the restriction that the sampling time step, $\Delta T$, is equal to the delay, $\tau_0$. In contrast, the 'reverse' direction might be a more appropriate model in situations where the discrete signal is the natural starting domain, for example a situation where I am only able to obtain a dataset comprising, say, 1-minute, or 5-minute, or 10-minute samples of AAPL's price. While the starting point is data-dependent, both the continuous ('forward') and discrete ('reverse') representations may provide insight into the advantages and limitations of the models under consideration. 
$\\[0.1in]$

## Unrolling the RNN

In the manner of broader neural network literature (see for example Goodfellow, Bengio & Courville (2016) and Grossberg (2013)), I use the term "cell" with reference to equations $\ref{5.37}$and/ or $\ref{5.39}$.  The cell is uninitialized at this point, meaning the sequence has been defined but not yet computed. The process of unrolling the RNN is that of specifying initial conditions for $h_t$ followed by numerical evaluation of $\ref{5.37}$, or $\ref{5.39}$, for the number of steps in the sequence. I have illustrated this process in figure 2:
$\\[0.1in]$

![Figure 2. Unrolling a RNN cell](fig_2.png)

Both equations $\ref{5.37}$ and $\ref{5.39}$ are recursive in the state signal $s \left[ n \right]$. Thus, as a result of repeated application of the recurrence relation as part of RNN unrolling, the state signal, $s \left[ n \right]$, at some value of the index, $n$, encompasses the contributions of the state signal $s \left[ k \right]$, and the input signal, $x \left[ k \right]$, for all indices, $k < n$, ending at $k = 0$, the start of the sequence.
$\\[0.1in]$

I define a step function, $g \left( x \right)$, thus:

\begin{equation*}
    g \left( x \right) =
    \begin{cases}
    1,  & x \geq 0 \\
    0,  & x < 0
    \end{cases} \label{5.41} \tag{5.41}
\end{equation*}
$\\[0.1in]$

where $0$ and $1$ are vectors comprising only elements equal to $0$ and $1$ respectively. 
$\\[0.1in]$

I define a unit delta function, $\delta \left( x \right)$ thus: 

\begin{align}
    \delta \left( x \right) = g \left( x \right) - u \left( x - 1 \right)
    \label{5.42} \tag{5.42} 
\end{align}
$\\[0.1in]$

where $\delta \left( x \right)$ is defined by being $1$ at $x = 0$ and $0$ otherwise.
$\\[0.1in]$

I use the unit step and unit delta functions to simulate a sequence of steps and compute the response of $h_t$, to see if a pattern emerges. For equation $\ref {5.37}$, with $b_h = 0$, the sequence for $h_t$ may be written in this way thus: 

\begin{align*}
    h_{t-1} &= 0 \\
    h_t &= W_x 1 \\
    h_{t+1} &= W_y J \left( W_x 1 \right) \\
    h_{t+2} &= W_y J \left( W_y J \left( W_x 1 \right) \right) \\
    h_{t+3} &= W_y J \left( W_y J \left( W_y J \left( W_x 1 \right) \right) \right) \\   
    h_{t+4} &= W_y J \left( W_y J \left( W_y J \left( W_y J \left( W_x 1 \right) \right) \right) \right) \\    
    \cdots
    \label{5.43} \tag{5.43}   
\end{align*}
$\\[0.1in]$

'Ground truth' is a term used in various fields of study, usually to describe empirical evidence, as opposed to information provided by inference. In deep learning, ground truth generally refers to the accuracy of a training dataset's classification for SL techniques (see for example Schmidhuber 2015, 2013) and thus may be used for, say, hypotheses testing. Ground truth values to the econometrician who writes her own code is an array of 'labels'. I denote a sequence of ground truth output values or labels, $l$, and let $t$ denote the number of time steps in the sequence, $l \left( T \right)$, where each time step equates to an observation, for example a tick-by-tick price observation in my HF datasets, and $T$ can be an arbitrarily large integer (the total number of samples in the HF training dataset, for example).
$\\[0.1in]$

I subdivide $l \left( T \right)$, into $N$ discrete segments comprising $k_n$ samples per segment, thus:  

\begin{align}
    l \left(T \right) = \sum^{N - 1}_{n = 0} l_n \left( t \right)
    \label{5.44} \tag{5.44} 
\end{align}
$\\[0.1in]$

The process of sub-dividing ground truth into $M$ non-overlapping segments is, in practice, often called 'windowing'. I therefore define a 'rectangular' window function, $w \left( x \right)$, taking the value $1$ inside the window and $0$ otherwise. In terms of the unit step function, $g \left( x \right)$ (equation $\ref{5.41}$), $w \left( x \right)$ may be defined thus:

\begin{align}
    w \left( x \right) = g \left( x \right) - g \left( x - k_0 \right)
    \label{5.45} \tag{5.45} 
\end{align}
$\\[0.1in]$

I may obtain an alternative 'sampling' definition or  $w_0 \left( x \right)$ by combining equations $\ref{5.42}$ and $\ref{5.45}$ thus:

\begin{align*}
    w_0 \left( x \right) &= l_x - l_{x-1} \\
    &+ l_{x-1} - l_{x-2} \\
    &+ l_{x-2} - l_{x-3} \\
    &+ ... + \\
    &+ l_{x-{k_0-2}} - l_{x-{k_0-1}}  \\
    &+ l_{x-{k_0-1}} - l_{x-{k_0}}  \\
    &= \sum^{K_0 - 1}_{k = 0} \delta \left[ x - k \right] 
    \label{5.46} \tag{5.46}   
\end{align*}
$\\[0.1in]$

In this way, from $\ref{5.46}$, I may 'window' the RNN:

\begin{align*}
    w \left( x \right) &= l_x - l_{x-{k_0}} \\ \\
    &= \sum^{M - 1}_{m = 0} \left[ \sum^{z \left( m \right) + K_m - 1}_{k = z \left( m \right)} \delta \left( x - k \right) \right] \label{5.47} \tag{5.47}  \\ \\
    &= \sum^{M - 1}_{m = 0} w_m \left( x \right) \label{5.48} \tag{5.48}   
\end{align*}
$\\[0.1in]$

Where:

\begin{equation*}
    z \left( m \right) =
    \begin{cases}
    \sum^{m-1}_{i=0} K_i,  & 1 \leq m \leq M - 1 \\
    0,  & m = 0
    \end{cases} \label{5.49} \tag{5.49}
\end{equation*}
$\\[0.1in]$

And:

\begin{align}
    w_m \left( x \right) = \sum^{z \left( m \right) + K_m - 1}_{k = z \left( m \right)} \delta \left( x - k \right)
    \label{5.50} \tag{5.50} 
\end{align}
$\\[0.1in]$

I make the following adjustments $v = k - z \left( m \right)$, $k = z \left( m \right) + l$, and $v \longrightarrow k$, in order to re-write equation $\ref{5.50}$ thus:

\begin{align}
    w_m \left( x \right) = \sum^{K_m - 1}_{k = 0} \delta \left( x - z_m - k \right)
    \label{5.51} \tag{5.51} 
\end{align}
$\\[0.1in]$

Equation $\ref{5.51}$ shows how each $w_m \left( x \right)$ comprises a a rectangular window of size $K_m$ samples. Thus, selecting a $K_m$-samples-long segment from ground truth equates to an element-wise multiplication of it by $w_m \left( x \right)$:

\begin{align*}
    l_m \left( x \right) &= w_m \left( x \right) \ast v l \left( x \right) \label{5.52} \tag{5.52} \\ \\
    &=
    \begin{cases}
    l \left( x \right),  &z_m \leq x \leq z_m + k_m -1 \\
    0,  &\mathrm{otherwise} 
    \end{cases} \label{5.53} \tag{5.53}
\end{align*}
$\\[0.1in]$

where $z \left( m \right)$ is given by equation $\ref{5.49}$. I define $\mathcal{A} \left( \langle y \left( x \right) \rangle \right)$ as an invertible linear transformation thus:

\begin{align}
    \langle v \left( x \right) \rangle = \mathcal{A} \left( \langle y \left( x \right) \rangle \right)
    \label{5.54} \tag{5.54} 
\end{align}
$\\[0.1in]$

And I define $\mathcal{L}$ as a loss or objective function (i.e. a measure of the 'cost' of $h_t$ and $y_t$ deviating from ground truth, thus:

\begin{align}
    \mathcal{L} \langle v \left( x \right) \rangle  \; , \; \langle l \left( x \right) \rangle    
    \label{5.55} \tag{5.55} 
\end{align}
$\\[0.1in]$

where $\langle v_x \rangle$ denotes the sequence of the observable output variables, $v_x$, and $\langle l_x \rangle$ denotes the sequence of ground truth output values, $l_x$. I may now write all parameters of the standard RNN system (i.e. equation $\ref{5.39}$, and recall again equation $\ref{3.3}$) thus:

\begin{align}
    \Theta \equiv \left\{ W_y, W_x, b_h \right\}  
    \label{5.56} \tag{5.56} 
\end{align}
$\\[0.1in]$

Following Sherstinsky (2020), I now explain the assumptions that form the basis of RNN unrolling. Firstly, given $\ref{5.39}$, parameterized by $\Theta$ (equation $\ref{5.56}$), I may assume a value of $\Theta$ exists at which the objective function $\mathcal{L}$ (equation $\ref{5.55}$) is close to an optimum. Secondly, I may also assume non-zero constants, $M$ and $K_m$, such that $K_m < X$, where $0 \leq m \leq M - 1$, and that ground truth, $l_x$, may be windowed (i.e. equation $\ref{5.53}$). In this way, I show that a single RNN cell, unrolled for $K_m$ steps, is computationally sufficient to obtain a value of $\Theta$ that optimizes $\mathcal{L}$ over a training dataset.
$\\[0.1in]$

The loss or objective function in equation $\ref{5.55}$ computes the error in the system's performance during training, validation, and testing phases and also tracks its generalization metrics with regard to actual application data during the inference phase (see section 6.1 Implementation). I state an assumption above that $\mathcal{L}$ may be optimized.  Thus, by implication, when $\mathcal{L}$ approaches an optimum, the observable output from the RNN system approximates ground truth:

\begin{align}
    \langle v \left( x \right) \rangle \sim \langle l \left( x \right) \rangle  
    \label{5.57} \tag{5.57} 
\end{align}
$\\[0.1in]$

I may now process the RNN's output using equation $\ref{5.52}$ to yield:

\begin{align*}
    v_m \left( x \right) &= w_m \left( x \right) \ast l \left( x \right) \label{5.58} \tag{5.58} \\ \\
    &=
    \begin{cases}
    y_x,  &z_m \leq x \leq z_m + k_m -1 \\
    0,  &\mathrm{otherwise} 
    \end{cases} \label{5.59} \tag{5.59}
\end{align*}
$\\[0.1in]$

where $z_m$ is given by equation $\ref{5.49}$.  
$\\[0.1in]$

Following equation $\ref{5.59}$, the individual output subsequences, $v_m \left( x \right)$ (equation $\ref{5.58}$), shall be non-zero only when $x$ is in the range $z_m \leq z \leq j_m + k_m -1$. And following the assumption that ground truth values are mutually independent, the loss function in equation $\ref{5.55}$ is separable.  The loss function may therefore be described via a set of $M$ independent segment-level components, thus:

\begin{align}
    \mathcal{L} \left( \langle v_x \rangle \; , \; \langle l_x \rangle  \right) =
    \mathcal{C} \left( \mathcal{L} \left[ \langle v_m \left( x \right) \rangle \; , \; \langle l_m \left( x \right) \rangle \right] \right)
    \label{5.60} \tag{5.60} 
\end{align}
$\\[0.1in]$

where $\mathcal{C}$ acts simply as a suitable combining function. As a result of equations $\ref{5.57}$ and $\ref{5.60}$ I may now write:
$\\[0.1in]$

\begin{align}
    \langle v_m \left( x \right) \rangle \sim \langle l_m \left( x \right) \rangle    
    \label{5.61} \tag{5.61} 
\end{align}
$\\[0.1in]$

Because $\mathcal{A} \left( \langle y_x \rangle \right)$ is invertible, I may write:

\begin{align}
    \langle y_m \left( x \right) \rangle = \mathcal{A}^{-1} \left( \langle v_m \left( x \right) \rangle \right)
    \label{5.62} \tag{5.62} 
\end{align}
$\\[0.1in]$

Recall that the output normalization function, $J \left( h_t \right)$, (equation $\ref{5.40}$) is invertible. I may therefore write for any $x$:

\begin{align}
    h_m \left( x \right) = J^{-1} \left( \langle v_m \left( x \right) \rangle \right)
    \label{5.63} \tag{5.63} 
\end{align}
$\\[0.1in]$

Element-wise multiplication of the input sequence, $x_t$ and sampling window, $w_m \left( x \right)$, yields:

\begin{align*}
    x_m \left( x \right) &= w_m \left( x \right) \ast x_m \left( x \right) \label{5.64} \tag{5.64} \\ \\
    &=
    \begin{cases}
    x_m \left( x \right),  &z_m \leq x \leq z_m + k_m - 1 \\
    0,  &\mathrm{otherwise} 
    \end{cases} \label{5.65} \tag{5.65}
\end{align*}
$\\[0.1in]$

I define $\mathcal{G} \left( \langle x_m \left( x \right) \rangle \leq x \leq z_m + k_m - 1 \right)$ as a transformation of the input sequence, $x$, into the output value $h$, so that the standard RNN definition in equation $\ref{5.39}$ for $k_m$ samples may be written thus:

\begin{align}
    \langle h_m \left( x \right) \rangle \leq x \leq z_m + k_{m - 1} 
    = \mathcal{G} \left( \langle x_m \left( x \right) \rangle \leq x \leq z_m + k_m - 1 \right)
    \label{5.66} \tag{5.66} 
\end{align}
$\\[0.1in]$

I may now substitute equations $\ref{5.65}$ and $\ref{5.66}$ into equation $\ref{5.39}$ to produce the RNN system equations for an individual window:

\begin{align*}
    h_m \left( x \right) &=
    \begin{cases}
    W_y y_m \left( x \right) + W_x x_m \left( x \right) + b_h,  &z_m  \leq x \leq z_m + k_m - 1 \\
    0,  &\mathrm{otherwise}
    \end{cases} \label{5.67} \tag{5.67} \\ \\
    y_m \left( x \right) &= 
    \begin{cases}
    J \left( h_m \left( x \right) \right),  &z_m  \leq x \leq z_m + k_m - 1 \\
    0,  &\mathrm{otherwise}
    \end{cases} \label{5.68} \tag{5.68} \\ \\
    h_m \left[ x = z_m - 1 \right]  &= 0 \label{5.69} \tag{5.69} \\ \\
    0 &\leq m \leq M - 1 \label{5.70} \tag{5.70}
\end{align*}
$\\[0.1in]$

I substitute $x \longrightarrow x + j \left( m \right)$ to shift $y_m \left( x \right)$, $h_m \left( x \right)$, and $x_m \left( x \right)$, by $- z_m$ samples, thus:
$\\[0.1in]$

\begin{align*}
    x_m \left( x \right) &\equiv x_m \left( x + z_m \right) \\ \\
    &=
    \begin{cases}
    x \left( x + z_m \right),  &z_m \leq x + j_m \leq j_m + k_m - 1 \\
    0,  &\mathrm{otherwise}
    \end{cases} \\ \\
    &=
    \begin{cases}
    x \left( x + z_m \right),  &0 \leq x \leq K_m - 1  \\
    0,  &\mathrm{otherwise}
    \end{cases} \label{5.71} \tag{5.71} \\ \\    
    y_m \left( x \right) &\equiv y_m \left( x + z_m \right) \\ \\
    h_m \left( x \right) &\equiv h_m \left( x + z_m \right) \label{5.72} \tag{5.72} \\ \\ 
    &=
    \begin{cases}
    W_y y_m \left( x + z_m - 1 \right) + W_x x_m \left( x + z_m \right) + b_h,  &z_m \leq x + z_m \leq z_m + k_m - 1 \\
    0,  &\mathrm{otherwise}
    \end{cases} \\ \\
    &=
    \begin{cases}
    W_y y_m \left( x \right) + W_x x_m \left( x \right) + b_h,  &0 \leq x \leq K_m - 1  \\
    0,  &\mathrm{otherwise}
    \end{cases} \label{5.73} \tag{5.73} \\ \\    
    h_m \left( x - 1 \right) &\equiv h_m \left[ x + z_m = z_m - 1 \right] = 0 \label{5.74} \tag{5.74}
\end{align*}
$\\[0.1in]$

And simplify thus:

\begin{align*}
    h_m \left( x - 1 \right)  &= 0 \label{5.75} \tag{5.75} \\ \\
    h_m \left( x \right) &=
    \begin{cases}
    W_y  y_m \left( x - 1 \right) + W_x x_m \left( x \right) + b_h,  &0 \leq x \leq K_m - 1 \\
    0,  &\mathrm{otherwise}
    \end{cases} \label{5.76} \tag{5.76} \\ \\
    y_m \left( x \right) &= 
    \begin{cases}
    J \left[ h_m \left( x \right) \right],  &0 \leq x \leq K_m - 1 \\
    0,  &\mathrm{otherwise}
    \end{cases} \label{5.77} \tag{5.77} \\ \\
    x_m \left( x \right) &= 
    \begin{cases}
    x \left[ x + z_m \right], &0 \leq x \leq K_m - 1 \\
    0, &\mathrm{otherwise}
    \end{cases} \label{5.78} \tag{5.78}
\end{align*}
$\\[0.1in]$

As per the standard RNN system, equation $\ref{5.39}$, the input sequence $x_m \left( x \right)$, is the only independent variable, unrolled for $K_m$ steps in equation $\ref{5.76}$. Along with the mutual independence of $h_m \left( x \right)$, this makes the RNN computationally generic for all windows.
$\\[0.1in]$

I now remove subscript $m$ from $h$ and $y$ to yield a final formulation of the RNN, unrolled for $K_m$ steps:

\begin{align*}
    h \left( x = -1 \right) &= 0 \label{5.79} \tag{5.79} \\ \\
    h \left[ n \right] &=
    \begin{cases}
    W_y y \left( x - 1 \right) + W_x x_m \left( x \right) + b_h,  &0 \leq x \leq K_m - 1 \\
    0,  &\mathrm{otherwise}
    \end{cases} \label{5.80} \tag{5.80} \\ \\
    y \left[ n \right] &= 
    \begin{cases}
    J \left( h_x \right),  &0 \leq x \leq K_m - 1 \\
    0,  &\mathrm{otherwise}
    \end{cases} \label{5.81} \tag{5.81} \\ \\
    x_m \left( x \right) &= 
    \begin{cases}
    x \left( x + z_m \right), &0 \leq x \leq K_m - 1 \\
    0, &\mathrm{otherwise} 
    \end{cases} \label{5.82} \tag{5.82} \\ \\
    0 &\leq m \leq M - 1 \label{5.83} \tag{5.83}
\end{align*}
$\\[0.1in]$

This formulation of the RNN, unrolled for $K_m$ steps, processes the windows, one at a time. Equation $\ref{5.79}$ initializes $h$, the cell output. Equation $\ref{5.82}$ then selects the input samples, following which equations $\ref{5.80}$ and $\ref{5.81}$ may be applied for $K_m$ steps, $0 \leq x \leq K_m - 1$. The same process may be applied recursively until all windows have been processed.  Mutual independence allows for computation of $h_m \left[ n \right]$, $y_m \left[ n \right]$, and $x_m \left[ n \right]$ to be carried out concurrently, as, for example, when using a GPU (see section 6.1 Implementation).
$\\[0.1in]$

## Vanishing gradients

In the preceding two sections, following the framework of Sherstinsky (2020), I have shown how equations $\ref{5.79}$ thru $\ref{5.83}$, together with equation $\ref{5.49}$, fully describe an unrolled RNN, in turn specified by equations $\ref{5.39}$ and $\ref{5.40}$. I proceed now to the process of 'training' a RNN in order to obtain its weights, focusing upon equations $\ref{5.80}$ and $\ref{5.81}$. RNN systems suffer from so-called vanishing (or exploding) gradients. In section 3.1, Deep learning history and topography, I explained that Hochreiter (1991) originally showed that with standard activation functions, cumulative back-propagated error signals either shrank rapidly, or grew out of bounds. I also noted that much subsequent research, particularly in the 1990’s and 2000’s, was motivated by this insight (see for example Pascanu, Mikolov & Bengio (2013), Hochreiter, Bengio, Paolo & Schmidhuber (2001), Hochreiter & Schmidhuber (1997)). I explained in section 3.1 the historic context whereby truncated unrolled RNN systems, such as equation $\ref{5.80}$, came to be  commonly trained using BP, adapted for sequences (see Werbos (1990, 1988) for detailed reviews). BP is quintessentially the repeated application of the chain rule of differentiation. 
$\\[0.1in]$

BP uses $x_m \left( x \right)$ and $y \left( x \right)$ in the training dataset to compute the parameters of the system, $\Theta$ (equation $\ref{5.56}$), in order to optimize a loss function, $L$. If GD, or alternative gradient algorithms (see for example Goodfellow, Bengio & Courville (2016) for a review), are used to simulate an optimum for $L$, the elements of $\frac {\partial L} {\partial \Theta}$ may be obtained via BP using the chain rule. Once the infinite RNN sequence in equation $\ref{5.39}$ is unrolled, the resulting system given in equation $\ref{5.80}$ becomes inherently stable and thus I may assume that the objective function $L$ may in turn take on the same form for all segments. I therefore now apply BP to $\ref{5.80}$. Consider that $L$ requires $y$ to compute the gradient of $L$ with respect to $y$, which I may call $\chi$, thus:

\begin{align*}
    \chi \left( x \right) \equiv \nabla_{y_x} L = \frac {\partial L} {\partial y_x}
    \label{5.84} \tag{5.84}
\end{align*}
$\\[0.1in]$

$y_x$ relies upon $h_x$, thus $h_x$ may also influence $L$, so that the  gradient of $L$ with respect to $h_x$, which I denote $\varphi$, ought be computed thus: 

\begin{align*}
    \varphi \left( x \right) \equiv \nabla_{h_x} L = \frac {\partial L} {\partial h_x}
    \label{5.85} \tag{5.85}
\end{align*}
$\\[0.1in]$

$\chi$ and $\varphi$ in equations $\ref{5.84}$ and $\ref{5.85}$ denote the the total partial derivative of the loss function, $L$, with respect to $y$ and $h$, respectively. $\chi$ and $\varphi$ constitute important auxiliary gradient sequences in the 'backward pass', which I deal with in detail subsequently in section 5.6 LSTM system derivatives. In practice, $L$ may be defined as the sum of the respective contributions of $y$, thus:
$\\[0.1in]$

\begin{align*}
    L = \sum^{K_m - 1}_{x = 0} L \left( y_x \right)
    \label{5.86} \tag{5.86}
\end{align*}
$\\[0.1in]$

Furthermore, $h_x$, at $x = k$, has been shown to influence $h_x$, at $x = k + 1$ (see for example Pascanu, Mikolov & Bengio (2013)). The dependency of $h_{x+1}$ on $h_x$ through $y_x$ may be shown by rewriting equation $\ref{5.80}$ thus:

\begin{align*}
    h_{x+1} &= W_y y_x + W_x x_m \left( x \right) + b_h \label{5.87} \tag{5.87} \\ \\ 
    y_x &= G \left( h_x \right) \\ \\
    y_{x+1} &= G \left( h_{t+1} \right) \\ \\
\end{align*}
$\\[0.1in]$

Thus, use of the chain rule in this way permits the description of the total partial derivative of $L$ with respect to $h_x$ and $y_x$:

\begin{align*}
    \chi \left( x \right) &= \frac {\partial L y_x} {\partial y_x} + W_y \varphi_x  \label{5.88} \tag{5.88} \\ \\
    \varphi \left( x \right) &= \chi_x \ast \frac {dJ \left( z \right)} {dz} \label{5.89} \tag{5.89} \\ \\
    &= \left( \frac {\partial L \left( y_x \right)} {\partial y_x} + W_y \varphi_{x+1} \right) \ast \frac {dG \left( z \right)} {dz} \label {5.90} \tag{5.90}
\end{align*}
$\\[0.1in]$

The sequence formed by the total partial derivative of $L$ with respect to $h_x$ and $y_x$, which is generally designated as the 'error gradient' in literature, requires $\ref{5.88}$ be initialized too, thus:

\begin{align*}
    \varphi \left( x = K_m \right) = 0 \label{5.91} \tag{5.91}
\end{align*}
$\\[0.1in]$

I may now write the derivatives of the model's parameters by applying the chain rule to $\ref{5.80}$ and using $\ref{5.90}$, thus:

\begin{align*}
    \frac {\partial L} {\partial \Theta} \left( x \right) &= \left\{ \frac {\partial L} {\partial W_y} \left( x \right), \frac {\partial L} {\partial W_x} \left( x \right), \frac {\partial E} {\partial b_h} \left( x \right) \right\} \label {5.92} \tag{5.92}  \\ \\
    \frac {\partial L} {\partial W_y} \left( x \right) &= \varphi_x y^T \left( x - 1 \right) \label {5.93} \tag{5.93}  \\ \\
    \frac {\partial L} {\partial W_x} \left( x \right) &= \varphi_x x^T_m \left( x \right) \label {5.94} \tag{5.94}  \\ \\
    \frac {\partial L} {\partial b_h} \left( x \right) &= \varphi_x  \label {5.95} \tag{5.95}  \\ \\
    \frac {dL} {d \Theta} &= \sum^{K_m - 1}_{x = 0} \frac {\partial L} {\partial \Theta} \left( x \right) \label {5.96} \tag{5.96}  \\ \\
\end{align*}
$\\[0.1in]$

The unrolled RNN cell shares the same set of model parameters, $\Theta$, for all steps. Consequently, the total derivative of $L$, with respect to $\Theta$, includes all steps, as illustrated by equation $\ref{5.96}$, which may now be used as part of optimization by GD. Furthermore, the quantities required to update the parameters of the system, $\Theta$, are, per equations $\ref{5.93}$, $\ref{5.94}$, and $\ref{5.95}$, directly proportional to $\varphi_x$. When training a RNN using BP, GD flows in the reverse direction of the index, $x$ and the fraction of $\varphi_x$ retained from BP is particularly important. This component of $L$ is responsible for adjusting the model's parameters, $\Theta$. I may expand the recursion in $\ref{5.90}$ to yield:

\begin{align*}
    \frac {\partial \varphi_x} {\partial \varphi_l} = \prod^l_{k = x + 1} W_y \ast \frac {dJ \left( z \right)} {dz}  \label {5.97} \tag{5.97}
\end{align*}
$\\[0.1in]$

Sherstinsky (2020) explains that the Jacobian matrix in $\ref{5.97}$, $\frac {\partial \varphi_x} {\partial \varphi_l}$, results from $l - x$ individual Jacobians, $W_y \ast \frac {dJ \left( z \right)} {dz}$. If the eigenvalues, $\mu_i$, of $W_y$, meet the stability requirement, $0 < \mu_i < 1$, then $\left\| W_y \right\| < 1$. Furthermore, $\left\| \frac {dJ \left( z \right)} {dz} \right\| < 1$, which follows the choice of normalization function (see equations $\ref{5.17}$, $\ref{5.18}$, $\ref{5.39}$, and $\ref{5.40}$). Thus:

\begin{align*}
    \left\| \frac {\partial \varphi_x} {\partial \varphi_l} \right\| &\sim \left( \left\| W_y \right\| \ast \left\| \frac {dJ \left( z \right)} {dz} \right\| \right)^{1 - x} \label {5.98} \tag{5.98} \\ \\ 
    &\sim \left\| W_y \right\|^{1 - x} \ast \left\| \frac {dJ \left( z \right)} {dz} \right\|^{1 - x} \approx 0  \label {5.99} \tag{5.99}
\end{align*}
$\\[0.1in]$

But should at least one eigenvalue of $W_y$ breach the stability requirement, $\left\| W_y \right\|^{1 - x}$ will grow exponentially, resulting in two possible outcomes: the vanishing, or exploding, gradients, discussed in section 3.1 deep learning history and topography. In the first case, $h_x$ grows, $y_x$ saturates, resulting in $\left\| \frac {\partial \varphi_x} {\partial \varphi_l} \right\| \approx 0$. The second case is rarer and results in $h_x$ biasing so that $\frac {dJ \left( z \right)} {dz} \neq 0$. If $x_m \left(x \right)$ keeps the RNN on this course for a large number of steps, $\left\| \frac {\partial \varphi_x} {\partial \varphi_l} \right\|$ grows, potentially overflowing. 
$\\[0.1in]$

I note in section 3.1 that much research, particularly in the 1990’s and 2000’s, was motivated by the above insight (see for example Pascanu, Mikolov & Bengio (2013), Hochreiter, Bengio, Paolo & Schmidhuber (2001), Hochreiter & Schmidhuber (1997)). The most effective solution so far (see for example Goodfellow, Bengio & Courville (2016)) is the long short-term memory (LSTM) cell architecture originally proposed by Hochreiter & Schmidhuber (1997).
$\\[0.1in]$

## The Long short-term memory recurrent neural network

Section 5.2 explains how the RNN, unrolled for $K_m$ steps, processes the parts, or segments, of it, one at a time. Equation $\ref{5.79}$ initializes $h$, the cell output. Equation $\ref{5.82}$ then selects the input samples, following which equations $\ref{5.80}$ and $\ref{5.81}$ are applied for $K_m$ steps, $0 \leq x \leq K_m - 1$. The same process is applied recursively until all the parts have been processed. For the rest of this paper, for brevity, I do not reproduce the segment notation of equation $\ref{5.80}$, so that henceforth all operations may be regarded to be on a fully unrolled cell, that is to say, whereby the index of the individual observations, $x$, travels all its steps, $0 \leq x \leq K_m - 1$, for all $x$, in $0 \leq m \leq M - 1$.
$\\[0.1in]$

I have shown in the earlier sections 5. Model, how, in the RNN system, $y_x$ is a normalized version of $h_x$. The weighted $y_x$ is fed back recursively as part of the update of $h_x$. The intimate relationship between $y_x$ and $h_x$ through this recursion directly impacts the gradient of $L$. The impact accumulates via training and results in the 'vanishing' (or more rarely 'exploding') gradient phenomena. The key insight of the long short-term memory recurrent neural network (LSTM RNN or LSTM) design is the incorporation of 'controls' directly into the network cell. The controls, which take the form of 'gates', may be trained in a manner that acts as a brake upon the gradient of the loss function with respect to $h$, i.e., the value that is proportional to parameter updates obtained from training by GD, and thus obviate the vanishing or exploding phenomena (see Schmidhuber (2015, 2013), Grossberg (2013), Perez-Ortiz, Gers, Eck, & Schmidhuber (2003), Hochreiter, Bengio, Paolo & Schmidhuber (2001), Gers, Schmidhuber & Cummins (2000), Hochreiter & Schmidhuber (1997)). The LSTM modifies the RNN cell in several important ways.
$\\[0.1in]$

Recall equation $\ref{5.37}$, which I now re-write thus: 

\begin{align*}
    h_x &= \mathcal{G}_h \left( h_{x - 1} \right) + \mathcal{G}_u \left( y_{x - 1}, x_x \right) \label{5.100} \tag{5.100} \\ \\
    y_x &= J \left( h_x \right) \label{5.101} \tag{5.101} \\ \\
    \mathcal{G}_h \left( h_{x - 1} \right) &= W_h h_{x-1}  \label{5.102} \tag{5.102} \\ \\
    \mathcal{G}_u \left( y_{x-1}, x_x \right) &= W_y y_{x-1} + W_x x_x + b_h  \label{5.103} \tag{5.103}
\end{align*}
$\\[0.1in]$

where $J \left( z \right)$ is the normalization function $\mathrm{tanh}$ (see equations $\ref{5.17}$ and $\ref{5.18}$). $\mathcal{G}_h \left( h_x - 1 \right)$ carries forward the value of $h$ from the previous step. $\mathcal{G}_u \left( r \left[ n - 1 \right], x \left[ n \right] \right)$, represents update (subscript '$u$') information, both normalized output, $y$ and input $x$ at the respective index $x$, plus the bias, $b_h$. In $\ref{5.100}$, $h$ combines the information in equal proportion at each step, but the proportions are rendered adjustable by the introduction of two new 'gate' signals, $f^g_x$, a so-called 'forget gate', and $i^g_x$, a so-called 'input gate', respectively, thus:

\begin{align*}
    h_x &= f^g_x \ast \mathcal{G}_h \left( h_{x-1} \right) + i^g_x \ast \mathcal{G}_u \left( y_{x-1}, x_x \right) \label{5.104} \tag{5.104} \\ \\
    0 &\leq f^g_x, i^g_x  \leq 1 \label{5.105} \tag{5.105}
\end{align*}
$\\[0.1in]$

Figure 3 illustrates the expansion of the canonical RNN system through the addition of a 'forget gate' $f^g_x$, to control the amount of $h$ retained from the previous step, and the addition of an 'input gate', $i^g_x$, to control the amount of input at the current step.
$\\[0.1in]$

![Figure 3. Introducing gates, i.e. the 'forget gate' and 'input gate', to the canonical RNN system](fig_3.png)

The forget and input gates in equations $\ref{5.104}$ and $\ref{5.105}$ provide the means by which the contributions of input $x$ and the carried foward state of the cell $h$ may be precisely controlled at every step. $W_h$ in $\ref{5.102}$ comprises positive fractions $\frac {1} {|a_{ii}|}$ on its main diagonal and because the elements of $f^g_x$ are also fractions, I may write:

\begin{align*}
    W_h = I \label{5.106} \tag{5.106}
\end{align*}
$\\[0.1in]$

in $f^g_x \ast W_h$, providing that the gate functions have parameters that are learned during training. 
$\\[0.1in]$

I may simplify $\ref{5.102}$ thus: 

\begin{align*}
    \mathcal{G}_s \left( h_{x-1} \right) = h_{x-1} \label{5.107} \tag{5.107}
\end{align*}
$\\[0.1in]$

so that equation $\ref{5.104}$ may be written:

\begin{align*}
    h_x &= f^g_x  \ast \mathcal{G}_h \left( h_{x-1} \right) + i^g_x \ast \mathcal{G}_u \left( y_{x-1}, x_x \right)  \\ \\
    &= f^g_x \ast y_{x-1} + i^g_x \ast \mathcal{G}_u \left( y_{x-1}, x_x \right) \label{5.108} \tag{5.108} 
\end{align*}
$\\[0.1in]$

With the result that the update term, $\mathcal{G}_u \left( y_{x-1}, x_x \right)$, is now controlled by the forget gate $f^g_x$. 
$\\[0.1in]$

Turning to the internal composition of $\mathcal{F}_u \left( y_{x-1}, x_x \right)$, equation $\ref{5.103}$ states that $y$ from the previous step and $x$ at the current step contribute equally. However, always using $W_y y_{x-1}$ is potentially problematic because when $i^g_x \sim 1$, $h_{x-1}$  $y_x$ become connected through $W_y$ and the associated normalization. Following $\ref{5.97}$, this makes the system susceptible to the vanishing gradient phenomenon.  
$\\[0.1in]$

This potential feedback path was eliminated by Hochreiter (1991) and Hochreiter & Schmidhuber (1997) by the introduction of a third gate, the output gate, which I denote $o^g_x$, thus: 

\begin{align*}
    l_x &= o^g_x \ast y_x   \label{5.109} \tag{5.109} \\ \\
    0 &\leq o^g_x \leq 1  \label{5.110} \tag{5.110} 
\end{align*}
$\\[0.1in]$

The output gate determines the cell's observable value at step $x$.
$\\[0.1in]$

Substituting $l_{x-1}$ for $y_x$ in $\ref{5.103}$ transforms it thus:

\begin{align*}
    \mathcal{G}_u \left( l_{x-1}, x_x \right) = W_y l_{x-1} + W_x x_x + b_h \label{5.111} \tag{5.111}
\end{align*}
$\\[0.1in]$

Figure 4 illustrates the expansion of the canonical RNN system through the further addition of the 'output gate' $o^g_x$, per equation $\ref{5.109}$, and its recursive relationship, equation $\ref{5.111}$.
$\\[0.1in]$

![Figure 4. The 'output gate' determines the amount of signal that becomes the cell's observable value at the current step](fig_4.png)

Equation $\ref{5.111}$ may be transposed by multiplying $x$ in $\ref{5.103}$ by the dedicated input gate $i^g_x$, thus:
$\\[0.1in]$

\begin{align*}
    \mathcal{G}_u \left( l_x, x_x \right) = W_y l_{x-1} + i^g_x x_x \ast W_x x_x + b_h \label{5.112} \tag{5.112}
\end{align*}
$\\[0.1in]$

Equations $\ref{5.109}$ and $\ref{5.112}$ show that the output gate, $o^g_x$, and the input gate, $i^g_x$, allow the update term, $\mathcal{G}_u \left( l_{x-1}, x_x \right)$, to contain aspects pf both output and input values.
$\\[0.1in]$

The value of $l_x \left[ n \right]$ is determined by the output $y_x$, in turn bounded by normalization, $J_d \left( z \right)$. To keep the same range and incorporate the input $i^g_x \ast x_x$, it is necessary that $\mathcal{G}_u \left( l_{x-1}, x_x \right)$, is also normalized using $J \left( z \right)$, to provide the 'update' $u_x$, thus: 

\begin{align*}
    u_x = J \left( \mathcal{G}_u \left( l_{x-1}, x_x \right) \right) \label{5.113} \tag{5.113}
\end{align*}
$\\[0.1in]$

The 'update' term in $\ref{5.108}$ may be transposed with $u_x$ in $\ref{5.113}$, to yield:

\begin{align*}
     h_x = f^g_x \ast h_{x-1} i^g_x \ast u_x \label{5.114} \tag{5.114} 
\end{align*}
$\\[0.1in]$

Equation $\ref{5.114}$ is part of a set of formulas that fully describe the LSTM network.  It shows that $h_x$ is a weighted combination of the value of the cell at the previous step and the cumulative information newly available at the present step.
$\\[0.1in]$

The core framework of the LSTM cell is thus described by the enhancement of the canonical RNN system via the addition of the forget, input, and output gates, illustrated in figure 5.
$\\[0.1in]$

![Figure 5. LSTM network showing $h_x$ as a weighted combination of $h_{x-1}$ and the aggregation of historical and novel update information available at the present step](fig_5.png)

I now define expressions for $f^g_x$, $i^g_x$, and $o^g_x$, as shown in figure 5.
$\\[0.1in]$

I assume that the LSTM will be trained with BP so that the gates are differentiable.
$\\[0.1in]$

Consider now the logistic function:

\begin{align*}
    J \left( z \right) &\equiv \sigma \left( z \right) \equiv \frac {1} {1 + e^{-z}} \label{5.115} \tag{5.115} \\ \\
    &= \frac {1 + \mathrm{tanh} \left( \frac {z} {2} \right) } {2} \label{5.116} \tag{5.116} \\ \\
    &= \frac {1 + J \left( \frac {z} {2} \right) } {2} \label{5.117} \tag{5.117}
\end{align*}
$\\[0.1in]$

as a transformation of the hyperbolic tangent, the normalization function, $J \left( z \right)$. 
$\\[0.1in]$

To determine fractional values for the forget, input, and output gates, $f^g_x$, $i^g_x$, and $o^g_x$ respectively, all preceding information may be utilized by the LSTM. 
$\\[0.1in]$

For both $f^g_x$, the forget gate, which determines the contribution to the cell's value, $h_{x-1}$, from the previous step, and the input gate, $i^g_x$, which determines the contribution to cell's value from the current step, the available information is $h_{x-1}$, $l_{x-1}$, and $x_x$.
$\\[0.1in]$

However, $o^g_x$ differs from the other two gates by having available to it $h_x$, $y_{x-1}$, and $x_x$. This results from $\ref{5.109}$, which ensures that $y_x$ is available and $\ref{5.101}$, which ensures $h_x$ is available. 
$\\[0.1in]$

The input to each gate may therefore be written thus:

\begin{align*}
    z_{f^g_x} &= W_{x_{f^g_x}} x_x + W_{h_{i^g_x}} h_{x-1} + b_{f^g_x} \label{5.118} \tag{5.118} \\ \\
    z_{i^g_x} &= W_{x_{i^g_x}} x_x + W_{h_{i^g_x}} h_{x-1} + b_{i^g_x} \label{5.119} \tag{5.119} \\ \\
    z_{o^g_x} &= W_{x_{o^g_x}} x_x + W_{h_{o^g_x}} h_x + b_{o^g_x} + \theta_{cr} \label{5.120} \tag{5.120} 
\end{align*}
$\\[0.1in]$

So that it follows that, as the weights are determined through training via BP,  the gate functions may be written thus:

\begin{align*}
    f^g_x &= J \left( z_{f^g_x} \right) \label{5.121} \tag{5.121} \\ \\
    i^g_x &= J \left( z_{i^g_x} \right) \label{5.122} \tag{5.122} \\ \\
    o^g_x &= J \left( z_{o^g_x} \right) \label{5.123} \tag{5.123} 
\end{align*}
$\\[0.1in]$

The gates constitute the seminal contribution of Hochreiter and Schmidhuber (1997) to DL.  The gates modulate training data through the LSTM system, thus enabling  the LSTM cell to interpret and obviate local optima and artificial boundaries that arise in the unrolling of the RNN and lead to the vanishing gradient phenomenon (Sherstinsky (2018, 2020), Schmidhuber (2013), Hochreiter & Schmidhuber (1997)). The gates robustify the LSTM system and enable it to handle phenomena in data and generate high quality output sequences. 
$\\[0.1in]$

## LSTM mechanism in detail

Sherstinsky (2020) posits that the LSTM cell fundamentally handles two potentially competing yet symbiotic objectives: (i) data; and (ii) data control (Sherstinsky, 2020). The data, $y_x$, $h_x$, and $x_x$, is in the normalized range $-1$ to $1$) and data control signals from the gates are also normalized in the range $0$ to $1$. 
$\\[0.1in]$

Element-wise multiplication of the data vectors by the control vectors provides the mechanism by which data is propagated optimally to nodes within the LSTM cell, so that, when, for example, the control signal is 0.6, 60% of data is passed on to the nodes. 
$\\[0.1in]$

Figure 6 illustrates the main operations of the LSTM cell that have been developed so far:
$\\[0.1in]$

![Figure 6. LSTM network cell](fig_6.png)

It may also be useful at this stage to summarize the notation that I have used to describe the LSTM cell so far. It may be summarized thus:
$\\[0.1in]$

$x$ denotes the index of an individual observation in a segment or window, $n = 0, ..., K - 1$
$\\[0.03in]$

$K$ denotes the number of observations
$\\[0.03in]$

$x_x$ denotes the input to the cell 
$\\[0.03in]$

$h_x$ denotes the output of the cell 
$\\[0.03in]$

$y_x$ denotes the normalized output of the cell 
$\\[0.03in]$

$J \left(\cdot \right)$ denotes a normalization function 
$\\[0.03in]$

$L$ denotes the loss function
$\\[0.03in]$

$a$ denotes an accumulation node
$\\[0.03in]$

$u$ denotes an update to the state of the cell 
$\\[0.03in]$

$g$ denotes a gate 
$\\[0.03in]$

$f^g_x$ denotes the forget gate
$\\[0.03in]$

$i^g_x$ denotes the input gate
$\\[0.03in]$

$o^g_x$ denotes the output gate 
$\\[0.03in]$

$W_{f^g_x}$ denotes the weight of the forget gate associated with the input (or output, and normalized output) respectively
$\\[0.03in]$

$W_{i^g_x}$ denotes the weight of the input gate associated with the input (or, again, the output, and normalized output) respectively
$\\[0.03in]$

$W_{o^g_x}$ denotes the weight of the output gate associated with the input (or, again as above, the output, and normalized output) respectively
$\\[0.03in]$

$b_{f^g_x}$ denotes the bias parameter, $b$, associated with the forget gate
$\\[0.03in]$

$b_{i^g_x}$ denotes the bias parameter, $b$, associated with the input gate
$\\[0.03in]$

$b_{o^g_x}$ denotes the bias parameter, $b$, associated with the output gate
$\\[0.03in]$

It is necessary to standardize data within the LSTM cell in order that all observations within the the training dataset input to it are mean $0$ and standard deviation $1$. This may be achieved in practice thus:

\begin{align*}
    \mu &= \frac {1} {K} \sum^{K - 1}_{k = 0} x_x \label{5.124} \tag{5.124} \\ \\
    \mathcal{K} &= \frac {1} {K - 1} \sum^{K - 1}_{k = 0} \left( x_x - \mu \right) {\left( x_x - \mu \right)}^T \label{5.125} \tag{5.125} \\ \\
    x_x &= {\left( \mathrm{diag} \left[ \sqrt{\mathcal{V}} \right] \right)}^{- 1} \left( x_x - \mu \right) \label{5.126} \tag{5.126} 
\end{align*}
$\\[0.1in]$

where $K$ denotes the number of observations, $mu$ denotes the sample mean, $\mathcal{K}$ denotes an auto-covariance matrix for the training dataset.
$\\[0.1in]$

In earlier sections of 5. Model I set out the rationale for use of the normalization function $J \left( \cdot \right)$ to output values between $0$ and $1$. The Sigmoidal, or logistic, function is widely used by researchers and practitioners to achieve this (see for example Chollet (2018), Goodfellow, Bengio & Courville (2016), and Grossberg (2013)) and may be computed thus:

\begin{align*}
    J \left( z \right) &\equiv \sigma \left( z \right) \\ \\
    \sigma \left( z \right) &\equiv \frac {1} {1 + e^{- z}}  = \frac {1 + \mathrm{tanh} \left( \frac {z} {2} \right)} {2} = \frac {1 + J \left( \frac {z} {2} \right)} {2} \label{5.127} \tag{5.127}
\end{align*}
$\\[0.1in]$

The hyperbolic tangent is also widely used by researchers and practitioners and may be computed thus:

\begin{align*}
    J \left( z \right) &\equiv \mathrm{tanh} \left( z \right) \\ \\ 
    \mathrm{tanh} \left( z \right) &= \frac {e^z - e^{- z}} {e^z + e^{- z}} = 2 \sigma \left( 2 z \right) - 1 = 2 J \left( 2 z \right) - 1 \label{5.122} \tag{5.128}
\end{align*}
$\\[0.1in]$

Finally, I may combine and/or reassemble the architecture of the LSTM cell described above into more formalized set of 15 parameter entities, thus: 
$\\[0.1in]$

$W_{x^i_x}$ denotes weights for the input, $x_x$, at the input gate.
$\\[0.03in]$

$W_{x^f_x}$ denotes weights for the state of the cell or cell output, $h_x$, at the input gate.
$\\[0.03in]$

$W_{x^o_x}$ denotes weights for the normalized out output, $y_x$, at the input gate.
$\\[0.03in]$

$b_{i^g_x}$ denotes the bias parameter, $b$, associated with the input gate
$\\[0.03in]$

$W_{h^i_x}$ denotes weights for the input, $x_x$, at the forget gate.
$\\[0.03in]$

$W_{h^f_x}$ denotes weights for the state of the cell or cell output, $h_x$, at the forget gate.
$\\[0.03in]$

$W_{h^o_x}$ denotes weights for the normalized out output, $y_x$, at the forget gate.
$\\[0.03in]$

$b_{f^g_x}$ denotes the bias parameter, $b$, associated with the forget gate
$\\[0.03in]$

$W_{y^i_x}$ denotes weights for the input, $x_x$, at the output gate.
$\\[0.03in]$

$W_{y^f_x}$ denotes weights for the state of the cell or cell output, $h_x$, at the output gate.
$\\[0.03in]$

$W_{y^o_x}$ denotes weights for the normalized out output, $y_x$, at the output gate.
$\\[0.03in]$

$b_{o^g_x}$ denotes the bias parameter, $b$, associated with the output gate
$\\[0.03in]$

$W_{u^a_x}$ denotes weights for the 'data update' accumulation node.
$\\[0.03in]$

$W_{u^l_x}$ denotes the weights connecting the cell state or output to the 'data update' accumulation node.
$\\[0.03in]$

$b_{u^a_x}$ denotes the bias parameter for the 'data update' accumulation node.
$\\[0.1in]$

The parameters of the LSTM network are generally denoted collectively by $\Theta$:

\begin{align*}
    \Theta \equiv \left\{ W_{x^i_x}, W_{x^f_x}, W_{x^o_x}, b_{i^g_x}, W_{h^i_x}, W_{h^f_x}, W_{h^o_x}, b_{f^g_x}, W_{y^i_x}, W_{y^f_x}, W_{y^o_x}, b_{o^g_x}, W_{u^a_x}, W_{u^l_x}, b_{u^a_x} \right\}
    \label{5.129} \tag{5.129}
\end{align*}
$\\[0.03in]$

$\Theta$ may also be usefully written thus:

\begin{align*}
    A = \begin{Bmatrix}
    W_{x^i_x} & W_{x^f_x} & W_{x^o_x} & b_{i^g_x} \\
    W_{h^i_x} & W_{h^f_x} & W_{h^o_x} & b_{f^g_x} \\
    W_{y^i_x} & W_{y^f_x} & W_{y^o_x} & b_{o^g_x} \\
    W_{u^a_x} & & W_{u^l_x} & b_{u^a_x}
    \end{Bmatrix}
    \label{5.130} \tag{5.130}
\end{align*}
$\\[0.1in]$

## The forward pass

The forward pass is a fundamental aspect of 'machine learning' and denotes the group of computations whereby the LSTM cell may be unrolled for $K$ steps in order to produce sequences of values or samples. The forward pass therefore shows how the values that characterize the step of the cell at index $x$, are based upon the values at $x - 1$. 
$\\[0.1in]$

The LSTM may be fully described by the following equations:

\begin{align*}
    a_{i^g_x} &=  W_{x^i_x} + W_{x^f_x} + W_{x^o_x} + b_{i^g_x} \label{5.131} \tag{5.131} \\ \\
    a_{f^g_x} &=  W_{h^i_x} + W_{h^f_x} + W_{h^o_x} + b_{f^g_x} \label{5.132} \tag{5.132} \\ \\
    a_{o^g_x} &=  W_{y^i_x} + W_{y^f_x} + W_{y^o_x} + b_{o^g_x} \label{5.133} \tag{5.133} \\ \\
    a_{i^g_x} &=  W_{u^a_x} + W_{u^l_x} + b_{u^a_x} \label{5.134} \tag{5.134} \\ \\
    u^a_x &= J \left( a_{i^g_x} \right) \label{5.135} \tag{5.135} \\ \\
    g_{i^g_x} &= J \left( a_{i^g_x} \right) \label{5.136} \tag{5.136} \\ \\
    g_{f^g_x} &= J \left( a_{f^g_x} \right) \label{5.137} \tag{5.137} \\ \\
    g_{o^g_x} &= J \left( a_{o^g_x} \right) \label{5.138} \tag{5.138} \\ \\
    h_x &= g_{f^g_x} \ast h_{x-1} + g_{i^g_x}] \ast u^a_x \label{5.139} \tag{5.139} \\ \\
    y_x &= J \left( h_x \right) \label{5.140} \tag{5.140} \\ \\
    l_x &= g_{o^g_x} \ast y_x \label{5.141} \tag{5.141} \\ \\
\end{align*}
$\\[0.1in]$

Figure 7 illustrates the LSTM cell, $\ref{5.131}$ - $\ref{5.141}$, unrolled over 4 steps.
$\\[0.1in]$

![Figure 7. The sequence of steps generated by unrolling a cell of the LSTM network](fig_7.png)

Figure 8 shows in full the operation of the LSTM cell and highlights specific functions as verticals. 
$\\[0.1in]$

![Figure 8. LSTM cell showing the stages of the system](fig_8.png)

## Backward propagation through time

Backward propagation, or backward propagation through time (both BP) is another fundamental process of ML and constitutes the mechanisms via which models are trained. Generally following the framework of Sherstinsky (2020), I describe the equations that are necessary for training the LSTM cell using BP. 
$\\[0.1in]$

To obtain update equations two supplementary gradient sequences that move in backward direction (hence 'backward propagation') may be computed.  The first: $\chi_x$, denotes the total partial derivative of the loss function, $L$, with respect $l_x$, whilst the second $\varphi_x$, denotes the total partial derivative of $L$ with respect to $h_x$ (recall, $\chi$ and $\varphi$ were introduced in section 5.3 Vanishing gradients (see for example equations $\ref{5.84}$ and $\ref{5.85}$)).
$\\[0.1in]$

$\chi_x$ and $\varphi_x$ bestow important architectural (implementation) benefits. Expressing each intra-cell total partial derivative in terms of $\chi_x$, instead of explicitly computing the total partial derivative of the objective function, $L$, with respect to each variable of the cell (Palangi, Deng, Shen, Gao, He, Chen, Song & Ward (2015) provide an example of this alternate approach), reduces the number of intermediate variables. As a result, the equations for the backward pass are more straightforward and architecturally more streamlined and easier to implement (see Abadi, Agarwal, Barham, Brevdo, Chen, Citro, Corrado, Davis, Dean, Devin, Ghemawat, Goodfellow, Harp, Irving, Isard, Jia, Jozefowicz, Kaiser, Kudlur, Levenberg, Mane, Monga, Moore, Murray, Olah, Schuster, Shlens, Steiner, Sutskever, Talwar, Tucker, Vanhoucke, Vasudevan, Viegas, Vinyals, Warden, Wattenberg, Wicke, Yu, & Zheng (2015), re TensorFlow's modularization of the backward pass, which codifies this approach, for example).
$\\[0.1in]$

The gradient sequences propagate backwards, in the direction opposite to that of the state signal, $h_x$. As a result, $\chi_x$ and $\varphi_x$ follow a backward-moving recursion and their respective values depend upon the values of the same quantities at the index $x + 1$. Once known, $\chi_x$ and $\varphi_x$ may be used to compute the total partial derivatives of $L$ with respect to accumulation nodes or intermediate gradient sequences, which I denote $\alpha_{f^g_x}$, $\alpha_{i^g_x}$, $\alpha_{o^g_x}$, and $\alpha_{u^a_x}$. 
$\\[0.1in]$

The LSTM cell takes the input signal at each step and computes the normalized output signal for all steps.  Other values are used internally within the cell, with the exception of $h_x$ and $l_x$, which naturally traverse steps within a sequence or window. $l_x$ may be transformed to provide the normalized output $y_x$. Similarly, the input, $h_x$, may be derived via transformation of the training dataset.  For example, in the empirical study, I use a weighted-mid-price (WMP) that is, in turn, computed from thoroughly cleansed and prepared high frequency stock and foreign exchanged price innovations. I also pay considerable attention in section 6.1 Implementation to 'windowing', the process whereby my high frequency input dataset is organized into segments of input samples that lie within selected 'windows', surrounding the given step and combined so as to enhance the overall system's 'attention' to context.
$\\[0.1in]$

I write the derivatives of the normalizing functions following $\ref{5.127}$ and $\ref{5.128}$, thus:

\begin{align*}
    \frac {dJ_c \left( z \right)} {dz} &= J_c \left( z \right) \left( 1 - J_c \left( z \right) \right) \label{5.142} \tag{5.142} \\ \\
    \frac {dJ_d \left( z \right)} {dz} &= 1 - {\left( J_d \left( z \right) \right)}^2 \label{5.143} \tag{5.143}
\end{align*}
$\\[0.1in]$

where the subscripts $d$ and $u$ denote data and control processes respectively.

I then define $\chi_x$ as the total partial derivative of the loss function, $L$, with respect to $l_x$ thus:

\begin{align*}
    \chi_x \equiv \nabla_{l_x} L = \frac {\partial L} {\partial l_x} \label{5.144} \tag{5.144} 
\end{align*}
$\\[0.1in]$

I may also define $\partial L$ with respect to three intermediate variables, $\rho_x$, $\gamma_x$, and $\alpha_x$, respectively, plus $\varphi_x$, thus:

\begin{align*}
    \rho_x &\equiv \nabla_{y_x} L = \frac {\partial L} {\partial y_x} \label{5.145} \tag{5.145} \\ \\
    \gamma_x &\equiv \nabla_{g_x} L = \frac {\partial L} {\partial g_x} \label{5.146} \tag{5.146} \\ \\
    \alpha_x &\equiv \nabla_{a_x} L = \frac {\partial L} {\partial a_x} \label{5.147} \tag{5.147} \\ \\
    \varphi_x &\equiv \nabla_{h_x} L = \frac {\partial L} {\partial h_x} \label{5.148} \tag{5.148} \\ \\
\end{align*}
$\\[0.1in]$

$\varphi_x$ is particularly important because, as the total partial derivative of the loss function with respect to the cell's state $h_x$ all parameter updates in the LSTM cell depend upon it.
$\\[0.1in]$

Equations $\ref{5.131}$ - $\ref{5.141}$ provide the BP equations by using $\ref{5.144}$ - $\ref{5.148}$ to apply the chain rule, thus:

\begin{align*}
    \chi_x &= {\left( \frac {\partial y_x} {\partial l_x} \right)}^T \left( \frac {\partial L} {\partial y_x} \right) + f_{\chi_x} \label{5.149} \tag{5.149} \\ \\
    \rho_x &= {\left( \frac {\partial l_x} {\partial y_x} \right)}^T \left( \frac {\partial L} {\partial l_x} \right) = \left( \nabla_{l_x} L \right) \ast o^g_x = \chi_x \ast o^g_x  \label{5.150} \tag{5.150} \\ \\
    \gamma_x &= \frac {\partial L} {\partial l_x} \frac {\partial l_x} {\partial o^g_x} = \left( \nabla_{l_x} L \right) \ast y_x = \chi_x \ast y_x  \label{5.151} \tag{5.151} \\ \\
    \alpha_x &= \gamma_x \ast \frac {\partial o^g_x} {\partial \alpha_{o^g_x}} = \gamma_x \ast \frac {dJ \left( z \right)} {dz} = \chi_x \ast y_x \ast \frac {dJ \left( z \right)} {dz} \label{5.152} \tag{5.152} \\ \\
    \varphi_x &= \rho_x \ast \frac {\partial y_x} {\partial h_x} + \frac {\partial \alpha_{o^g_x}} {\partial h_x} \alpha_{o^g_x} + r_{\varphi_{x+1}} \label{5.153} \tag{5.153} \\ \\
    &= \rho_x \ast {J \left( z \right)} {dz} + W_{h_{o^g_x}} \alpha_{o^g_x} + r_{\varphi_{x+1}} \label{5.154} \tag{5.154} \\ \\
    &= \chi_x \ast o^g_x \ast \frac {dJ \left( z \right)} {dz} + W_{s_{o^g_x}} \alpha_{o^g_x} + r_{\varphi_{x+1}} \label{5.155} \tag{5.155} \\ \\
    \alpha_{f^g_x} &= \varphi_x \ast \frac {\partial h_x} {\partial f^g_x} \ast \frac {\partial f^g_x} {\partial \alpha_{f^g_x}} = \varphi_x \ast h_{t-1} \ast \frac {dJ \left( z \right)} {dz} \label{5.156} \tag{5.156} \\ \\
    \alpha_{i^g_x} &= \varphi_x \ast \frac {\partial h_x} {\partial i^g_x} \ast \frac {\partial i^g_x} {\partial \alpha_{i^g_x}} = \varphi_x \ast u_x \ast \frac {dJ \left( z \right)} {dz} \label{5.157} \tag{5.157} \\ \\
    \alpha_{u^a_x} &= \varphi_x \ast \frac {\partial h_x} {\partial u_x} \ast \frac {dJ \left( z \right)} {dz} = \varphi_x \ast i^g_x \ast \frac {dJ \left( z \right)} {dz} \label{5.158} \tag{5.158}
\end{align*}
$\\[0.1in]$

where:

\begin{align*}
    r_{\chi_{x+1}} &= W_{l_{i^g_x}} \alpha_{{i^g_x}_{x+1}} + W_{l_{f^g_x}} \alpha_{{f^g_x}_{x+1}} + W_{l_{o^g_x}} \alpha_{{o^g_x}_{x+1}} + W_{l_{u^a_x}} \alpha_{{u^a_x}_{x+1}} \label{5.159} \tag{5.159} \\ \\
    r_{\varphi_{x+1}} &= W_{h_{i^g_x}} \alpha_{{i^g_x}_{x+1}} + W_{h_{i^g_x}} \alpha_{{f^g_x}_{x+1}} + {f^g_x}_{x+1} \ast \varphi_{x+1} \label{5.160} \tag{5.160} \\ \\
\end{align*}
$\\[0.1in]$

are fractions of the total derivative of $L$ with respect to $y_x$ and $h_x$ at $x+1$.
$\\[0.1in]$

The total partial derivatives of $L$ with respect to $\Theta$, i.e. $\ref{5.129}$, are proportional to $\ref{5.152}$, $\ref{5.156}$, $\ref{5.157}$, and $\ref{5.158}$. Thus, from $\ref{5.131}$ - $\ref{5.141}$, I have:

\begin{align*}
    \frac {\partial L} {\partial W_{x_{i^g_x}}} &= \alpha_{i^g_x} x^T_x \label{5.161} \tag{5.161} \\ \\
    \frac {\partial L} {\partial W_{s_{i^g_x}}} &= \alpha_{i^g_x} h^T_{x-1} \label{5.162} \tag{5.162} \\ \\
    \frac {\partial L} {\partial W_{l_{i^g_x}}} &= \alpha_{i^g_x} l^T_{x-1} \label{5.163} \tag{5.163} \\ \\
    \frac {\partial L} {\partial b_{i^g_x}} &= \alpha_{i^g_x} \label{5.164} \tag{5.164} \\ \\
    \frac {\partial L} {\partial W_{x_{f^g_x}}} &= \alpha_{f^g_x} x^T_x \label{5.165} \tag{5.165} \\ \\
    \frac {\partial L} {\partial W_{h_{f^g_x}}} &= \alpha_{f^g_x} h^T_{x-1} \label{5.166} \tag{5.166} \\ \\
    \frac {\partial L} {\partial W_{l_{f^g_x}}} &= \alpha_{f^g_x} l^T_{x-1} \label{5.167} \tag{5.167} \\ \\
    \frac {\partial L} {\partial b_{f^g_x}} &= \alpha_{f^g_x} \label{5.168} \tag{5.168} \\ \\
    \frac {\partial L} {\partial W_{x_{o^g_x}}} &= \alpha_{o^g_x} x^T_x \label{5.169} \tag{5.169} \\ \\
    \frac {\partial L} {\partial W_{h_{o^g_x}}} &= \alpha_{o^g_x} h^T_x \label{5.170} \tag{5.170} \\ \\
    \frac {\partial L} {\partial W_{l_{o^g_x}}} &= \alpha_{o^g_x} l^T_{x-1} \label{5.171} \tag{5.171} \\ \\
    \frac {\partial L} {\partial b_{o^g_x}} &= \alpha_{o^g_x} \label{5.172} \tag{5.172} \\ \\
    \frac {\partial L} {\partial W_{x_{u^a_x}}} &= \alpha_{u^a_x} h^T_x \label{5.173} \tag{5.173} \\ \\
    \frac {\partial L} {\partial W_{l_{u^a_x}}} &= \alpha_{u^a_x} l^T_{x-1} \label{5.174} \tag{5.174} \\ \\
    \frac {\partial L} {\partial b_{u^a_x}} &= \alpha_{u^a_x} \label{5.175} \tag{5.175} \\ \\
\end{align*}
$\\[0.1in]$

Re-arranged to the form of $\ref{5.130}$, $L$ with respect to $\Theta$, at index $x$, may be written thus:

\begin{align*}
    \frac {\partial L} {\partial \Theta} \left( x \right) = \begin{Bmatrix}
    \frac {\partial L} {\partial W_{x_{i^g_x}}} & \frac {\partial L} {\partial W_{s_{i^g_x}}} & \frac {\partial L} {\partial W_{l_{i^g_x}}} & \frac {\partial L} {\partial b_{i^g_x}} \\
    \frac {\partial L} {\partial W_{x_{f^g_x}}} & \frac {\partial L} {\partial W_{h_{f^g_x}}} & \frac {\partial L} {\partial W_{l_{f^g_x}}} & \frac {\partial L} {\partial b_{f^g_x}} \\
    \frac {\partial L} {\partial W_{x_{o^g_x}}} & \frac {\partial L} {\partial W_{h_{o^g_x}}} & \frac {\partial L} {\partial W_{l_{o^g_x}}} & \frac {\partial L} {\partial b_{o^g_x}} \\
    \frac {\partial L} {\partial W_{x_{u^a_x}}} & & \frac {\partial L} {\partial W_{l_{u^a_x}}} & \frac {\partial L} {\partial b_{u^a_x}}
    \end{Bmatrix}
    \label{5.176} \tag{5.176}
\end{align*}
$\\[0.1in]$

When unrolling a LSTM cell for $K$ observations, i.e. a full window of the training dataset, $\Theta$, is shared by all steps because $\Theta$ applies to the cell as a whole. As a result $L$ aggregates all steps in the window, thus:

\begin{align*}
    \frac {d L} {d \Theta} = \sum^{K = 1}_{n = 0} \frac {\partial L} {\partial \Theta} \left( x \right)
    \label{5.177} \tag{5.177}
\end{align*}
$\\[0.1in]$

$\ref{5.177}$ may now be used by GD and in the empirical setting $\ref{5.177}$ is computed for the window and then supplied to the GD optimization algorithm.  
$\\[0.1in]$

## Gradient descent

In the preceding sections I have been able to show, by generally following the framework set out by Sherstinsky (2020), why the introduction of the gates Hochreiter & Schmidhuber (1997) makes the LSTM network better suited to GD training than the standard RNN. GD works best when the elements of $\frac {\partial L} {\partial \Theta} \left( x \right)$ are well-behaved numerically. By implication, $\alpha_{f^g_x}$,  $\alpha_{i^g_x}$, $\alpha_{o^g_x}$, and $\alpha_{u^a_x}$, along with, $\chi_x$ and $\varphi_x$, are required to be robust over the range of $K$ steps. 
$\\[0.1in]$

Expanding $\ref{5.155}$ yields:

\begin{align*}
    \varphi_x & = \chi_x \ast o^g_t \ast \frac {dJ \left( z \right)} {dz} + W_{h_{o^g_x}} \chi_x  \ast y_x \ast \frac {dJ \left( z \right)} {dz} 
    + r_{\varphi_{x+1}} \label{5.178} \tag{5.178} \\ \\
    &= \left[{\left( \frac {\partial y_x} {\partial l_x} \right)}^T \left( \frac {\partial L} {\partial y_x} \right) + r_{\chi_{x+1}} \right] 
    \ast o^g_x \frac {dJ \left( z \right)} {dz} \\ 
    &+ W_{h_{o^g_x}} \left[{\left( \frac {\partial y_x} {\partial l_x} \right)}^T \left( \frac {\partial L} {\partial y_x} \right) + r_{\chi_{t+1}} \right] \ast y_x \ast \frac {dJ \left( z \right)} {dz} + r_{\varphi_{x+1}} 
    \label{5.179} \tag{5.179}
\end{align*}
$\\[0.1in]$

It follows from $\ref{5.159}$ and $\ref{5.160}$ that $\chi_x$ and $\varphi_x$ depend upon $\varphi_{x+1}$. I therefore repeat the approach taken in section 5.3 'Vanishing gradients', shift indexes $x \longrightarrow k - 1$, and apply the chain rule to $\ref{5.179}, to yield:

\begin{align*}
    \frac {\partial \varphi_{k - 1}} {\partial \varphi_k} 
    & = \left( \frac {\partial \varphi_{k-1}} {\partial r_{\chi_k}} \right) \left( \frac {\partial r_\chi} {\partial \varphi_k} \right)
    + \left( \frac {\partial \varphi_{k-1}} {\partial r_{\varphi_k}} \right) \left( \frac {\partial r_{\varphi_k}} {\partial \varphi_k} \right)
    \label{5.180} \tag{5.180} \\ \\
    & = \left( \frac {\partial \varphi_{k-1}} {\partial r_{\chi_k}]} \right) 
    \left\{
    \left( \frac {\partial r_{\chi_k}} {\partial \alpha_{i^g_k}} \right) \left( \frac {\partial \alpha_{i^g_k}} {\partial \varphi_k} \right) +
    \left( \frac {\partial r_{\chi_k}} {\partial \alpha_{f^g_k}} \right) \left( \frac {\partial \alpha_{f^g_k}} {\partial \varphi_k]} \right) +
    \left( \frac {\partial r_{\chi_k}} {\partial \alpha_{u^a_k}} \right) \left( \frac {\partial \alpha_{u^a_k}} {\partial \varphi_k]} \right)    
    \right\} \\
    & + \; \left( \frac {\partial \varphi_{k-1}} {\partial r_{\varphi_k}} \right) 
    \left\{
    \left( \frac {\partial r_{\varphi_k}} {\partial \alpha_{i^g_k}} \right) \left( \frac {\partial \alpha_{i^g_k}} {\partial \varphi_k} \right) +
    \left( \frac {\partial r_{varphi_k}} {\partial \alpha_{f^g_k}} \right) \left( \frac {\partial \alpha_{f^g_k}} {\partial \varphi_k} \right) +
    \mathrm{diag} \bigg[ f^g_k \bigg]
    \right\}    
    \label{5.181} \tag{5.181} \\ \\
    &= \left( 
    \mathrm{diag} \bigg[ {o^g_k}_{k-1} \ast \frac {dJ \left( z \right)} {dz} \bigg]  
    + W_{h_{o^g_k}} \mathrm{diag} \bigg[ y_{k-1} \ast \frac {dJ \left( z \right)} {dz} \bigg] 
    \right) \\
    &\times \left\{ 
    W_{l_{i^g_k}} \mathrm{diag} \bigg[ u_k \bigg] \ast \frac {dJ \left( z \right)} {dz} \bigg]  
    + W_{l_{f^g_k}} \mathrm{diag} \bigg[ h_{k-1} \ast \frac {dJ \left( z \right)} {dz} \bigg] 
    + W_{l_{u^a_k}} \mathrm{diag} \bigg[ i^g_k \ast \frac {dJ \left( z \right)} {dz} \bigg] 
    \right\} \\
    &+ \left(
    W_{h_{i^g_k}} \mathrm{diag} \bigg[ u_k \ast \frac {dJ \left( z \right)} {dz} \bigg]
    + W_{h_{f^g_k}} \mathrm{diag} \bigg[ h_{k-1} \ast \frac {dJ \left( z \right)} {dz} \bigg]
    + \mathrm{diag} \bigg[ f^g_k \bigg]
    \right)
    \label{5.182} \tag{5.182} \\ \\
    &=  \mathrm{diag} \bigg[ u_n \ast \frac {dJ} {dz} \bigg]
    \times \left\{
    W_{l_{f^g_k}} \left( 
    \mathrm{diag} \bigg[ o^g_k \ast \frac {dJ \left( z \right)} {dz} \bigg]
    +  W_{h_{o^g_k}} \mathrm{diag} \bigg[y_{k-1} \ast \frac {dJ \left( z \right)} {dz} \bigg]
    \right) + W_{h_{i^g_k}} 
    \right\} \\
    &+  \mathrm{diag} \bigg[ h_{k-1} \ast \frac {dJ \left( z \right)} {dz} \bigg]
    \times \left\{
    W_{l_{f^g_k}} \left( 
    \mathrm{diag} \bigg[ o^g_k \ast \frac {dJ \left( z \right)} {dz} \bigg]
    +  W_{h_{o^g_k}} \mathrm{diag} \bigg[y_{k-1} \ast \frac {dJ \left( z \right)} {dz} \bigg]
    \right) + W_{h_{f^g_k}} 
    \right\} \\
    &+  \mathrm{diag} \bigg[ i^g_k \ast \frac {dJ \left( z \right)} {dz} \bigg]
    \times \left\{
    W_{l_{a^u_k}} \left( 
    \mathrm{diag} \bigg[ o^g_k \ast \frac {dJ \left( z \right)} {dz} \bigg]
    + W_{h_{o^g_k}} \mathrm{diag} \bigg[ y_{k-1} \ast \frac {dJ \left( z \right)} {dz} \bigg]
    \right) \right\} \mathrm{diag} \bigg[ f^g_k \bigg]
    \label{5.183} \tag{5.183} \\ \\
    &= \mathcal{J} \left( k - 1, k ; \Theta \right) + \mathrm{diag} \bigg[ f^g_k \bigg]
    \label{5.184} \tag{5.184} 
\end{align*}
$\\[0.1in]$

where $\Theta = \left\{ W_{h_{i^g_x}}, W_{l_{i^g_x}}, W_{h_{f^g_k}}, W_{l_{f^g_k}}, W_{h_{o^g_k}}, W_{l_{u^a_k}} \right\}$ and $\mathcal{J} \left( k - 1, k ; \Theta \right)$ comprise all the terms in $\frac {\partial \varphi_{k-1}} {\partial \varphi_k}$, with the exception of $\mathrm{diag} \bigg[ f^g_k \bigg]$, and $\mathrm{diag} \bigg[ \mathcal{v} \bigg]$ denotes a diagonal matrix in which vector $\mathcal{v}$ occupies the main diagonal.
$\\[0.1in]$

To train the LSTM, I process $\frac {\partial \varphi_{k-1}} {\partial \varphi_k}$ from index $x$ to $z \leq K - 1$, where $z \gg n$, thus:
$\\[0.1in]$

\begin{align*}
    \frac {\partial \varphi_x} {\partial \varphi_z} = \prod^z_{k = x + 1} \frac {\partial \varphi_{k-1}} {\partial \varphi_k} 
    \label{5.185} \tag{5.185}
\end{align*}
$\\[0.1in]$

Recall the seminal contribution of the LSTM network, namely obviating 'vanishing gradients' (I continue to assume for convenience that the far less common problem of exploding gradients may be dealt with separately). I may take the value of $L$ with respect to $h_z$, so that $\ref{5.185}$ in fact determines how much of the gradient sequence did not 'vanish' during training and instead was held by $\varphi_x$ and thus available to update the LSTM's parameters.  
$\\[0.1in]$

# Conclusion

In this post I provide a thorough derivation of the LSTM network, based upon the framework set out by Sherstinsky (2018, 2020), a step generally skipped in finance deep learning literature featuring the LSTM.
$\\[0.1in]$

# References

Ahmed, A., Smola, A., Wang, Y., Xing, E.P., Zaheer, M. & Zheng, X. (2018). State space LSTM models with particle MCMC inference. arXiv:1711.11179v1.
$\\[0.05in]$

Almosova, A. & Andresen, N. (2019). Nonlinear inflation forecasting with recurrent neural networks. Working Paper.
$\\[0.05in]$

Amari, S. (1967). A theory of adaptive pattern classifiers. IEEE Trans. EC, 16, 3 299–307.
$\\[0.05in]$

Archer, E., Park, I.M., Buesing, L., Cunningham, J. & Paninski, L. (2015). Black box variational inference for state space models. arXiv:1511.07367.
$\\[0.05in]$

Arneric, J., Poklepovic, T. & Aljinovic, Z. (2014). GARCH based artificial neural networks in forecasting conditional variance of stock returns. Croatian Operational Research Review, 5, 329–343.
$\\[0.05in]$

Balkin, S.D. & Ord, J.K. (2000). Automatic neural network modeling for univariate time series. International Journal of Forecasting, 16(4), 509-515.
$\\[0.05in]$

Bandara, K., Bergmeir, C. & Smyl, S. (2017). Forecasting across time series databases using recurrent neural networks on groups of similar series:
A clustering approach. arXiv:1710.03222.
$\\[0.05in]$

Bayer, J. & Osendorfer, C. (2014). Learning stochastic recurrent networks. arXiv:1411.7610.
$\\[0.05in]$

Bengio, Y., Lamblin, P., Popovici, D. & Larochelle, H. (2007). Greedy layer-wise training of deep networks. Advances in Neural Information Processing Systems (NIPS’06), 153-160.
$\\[0.05in]$

Bengio, Y., Simard, P., & Frasconi, P. (1994). Learning long-term dependencies with gradient descent is difficult. IEEE Transactions on Neural Networks, 5(2), 157–166.
$\\[0.05in]$

Bishop, C.M. (1994). Mixture density networks. Technical report, 1994.
$\\[0.05in]$

Borovkova, S. & Tsiamas, I. (2018). An ensemble of LSTM neural networks for high frequency stock market classification. Journal of Forecasting, 38, 600-619.
$\\[0.05in]$

Boulanger-Lewandowski, N., Bengio, Y. & Vincent, P. (2012). Modeling temporal dependencies in high-dimensional sequences: Application to polyphonic music generation and transcription. In ICML, 2012.
$\\[0.05in]$

Butcher, J.C. (2003). Numerical methods for ordinary differential equations. John Wiley & Sons, New York.
$\\[0.05in]$

Bryson, A. E. (1961). A gradient method for optimizing multi-stage allocation processes. In Proc. Harvard Univ. Symposium on digital computers and their applications.
$\\[0.05in]$

Bryson, Jr., A. E. & Denham, W. F. (1961). A steepest-ascent method for solving optimum programming problems. Technical Report BR-1303, Raytheon Company, Missle and Space Division.
$\\[0.05in]$

Bryson, A. & Ho, Y. (1969). Applied optimal control: optimization, estimation, and control. Blaisdell Pub. Co.
$\\[0.05in]$

Bucci, A. (2019). Realized volatility forecasting with neural networks. MPRA Paper No. 95443. Online at https://mpra.ub.uni-muenchen.de/95443/.
$\\[0.05in]$

Caley, J.A., Lawrance, N.R.J. & Hollinger, G.A. (2017). Deep networks with confidence bounds for robotic information gathering.
$\\[0.05in]$

Carr, P., Wu, L. & Zhang, Z.B. (2020). Using machine learning to predict realized variance. Journal of Investment Management, Vol. 18, No. 2, 1-16.
$\\[0.05in]$

Chang, C.H., Rampasek, L. & Goldenberg, A. (2017). Dropout feature ranking for deep learning models. arXiv:1712.08645.
$\\[0.05in]$

Che, Z.P., Purushotham, S., Cho, K.Y., Sontag, D., & Liu, Y. (2018). Recurrent neural networks for multivariate time series. Scientific reports, 8(1), 6085.
$\\[0.05in]$

Cho, K.H., Chung, J.Y., Gulcehre, C. & Bengio, Y. (2014). Empirical evaluation of gated recurrent neural networks on sequence modeling. arXiv preprint arXiv:1412.3555, 2014.
$\\[0.05in]$

Chung, J.Y., Kastner, K., Dinh, L., Goel, K., Courville, A. & Bengio, Y. (2015). A recurrent latent variable model for sequential data. In NIPS, 2015.
$\\[0.05in]$

Cinar, Y.G., Mirisaee, H., Goswami, P., Gaussier, E, Ait-Bachir, A. & Strijov, F.V. (2017). Time series forecasting using RNNs: an extended attention mechanism to model periods and handle missing values. CoRR, abs/1703.10089.
$\\[0.05in]$

Connor, J.T., Douglas Martin, R. & Atlas, L.E. (1994). Recurrent neural networks and robust time series prediction. IEEE transactions on neural networks, Vol. 5, No. 2, 240-254.
$\\[0.05in]$

de Vries, B. & Principe, J.C. (1991). A theory for neural networks with time delays. In Lippmann, R.P., Moody, J.E. & Touretzky, D.S. (Eds), Advances in Neural Information Processing Systems 3, Morgan Kaufmann. 162–168.
$\\[0.05in]$

Diebold, F.X., Ghysels, E., Mykland, P. & Zhang, L. (2019). Big data in dynamic predictive econometric modeling. Editorial in Journal of Econometrics, 212, 1-3.
$\\[0.05in]$

Director, S. W. & Rohrer, R. A. (1969). Automated network design - the frequency-domain case. IEEE Trans. Circuit Theory, CT, 16, 330–337.
$\\[0.05in]$

Dixon, M., Klabjan, D. & Bang, J.H. (2015). Implementing deep neural networks for fnancial market prediction on the Intel Xeon Phi. In: roceedings of the 8th Workshop on High Performance Computational Finance. 1-6.
$\\[0.05in]$

Dixon, M., Klabjan, D. & Bang, J.H. (2017). Classification-based financial markets prediction using deep neural networks. Algorithmic Finance, 6, 67–77
$\\[0.05in]$

Donaldson, G.R. & Kamstra, M. (1996a). A new dividend forecasting procedure that rejects bubbles in asset prices. Review of Financial Studies, 8, 333–383.
$\\[0.05in]$

Donaldson, G.R. & Kamstra, M. (1996b). Forecast Combining with Neural Networks. Journal of Forecasting, 15, 49–61.
$\\[0.05in]$

Donaldson, G.R. & Kamstra, M. (1997). An artificial neural network-GARCH model for international stock return volatility. Journal of Empirical Finance 4, 17–46.
$\\[0.05in]$

Downey, C., Hefny, A. & Gordon, G. (2017). Practical learning of predictive state representations. arXiv preprint arXiv:1702.04121.
$\\[0.05in]$

Dreyfus, S. E. (1962). The numerical solution of variational problems. Journal of Mathematical Analysis and Applications, 5, 30–45.
$\\[0.05in]$

Faraway, J. & Chatfield, C. (1998). Time series forecasting with neural networks: a comparative study using the air line data. Journal of the Royal Statistical Society: Series C (Applied Statistics), 47(2), 231-250.
$\\[0.05in]$

Fernandes, M., Medeiros, M.C., Scharth, M., 2014. Modeling and predicting the CBOE market volatility index. Journal of Banking & Finance, 40, 1–10.
$\\[0.05in]$

Fischer, T. & Krauss, C. (2017). Deep learning with long short term memory networks for financial market predictions. FAU Discussion Papers in Economics, No. 11/2017, Friedrich-Alexander-Universität Erlangen-Nürnberg, Institute for Economics, Erlangen.
$\\[0.05in]$

Flunkert, V., Salinas, D. & Gasthaus, J. (2017). Deepar: Probabilistic forecasting with autoregressive recurrent networks. arXiv preprint arXiv:1704.04110, 2017.
$\\[0.05in]$

Fraccaro, M., Sønderby, S.K., Paquet, U. & Winther, O. (2016). Sequential neural models with stochastic layers. In NIPS, 2016.
$\\[0.05in]$

Graves, A., Liwicki, M., Fernandez, S., Bertolami, R., Bunke, H. & Schmidhuber, J. (2009). A Novel connectionist system for improved unconstrained handwriting recognition. IEEE Transactions on Pattern Analysis and Machine Intelligence, 31, 5.
$\\[0.05in]$

Gers, F. A., Schmidhuber, J., & Cummins, F. (2000). Learning to forget: Continual prediction with LSTM. Neural Computation, 12, 2451–2471.
$\\[0.05in]$

Gers, F.A., Schraudolph, N.N. & Schmidhuber, J. (2002). Learning precise timing with LSTM recurrent networks. Journal of machine learning research, 3, 115-143.
$\\[0.05in]$

Ghahramani, Z. & Roweis, S.T. (1999). Learning nonlinear dynamical systems using an EM algorithm. In NIPS, 1999.
$\\[0.05in]$

Giles, C.L., Lawrence, S. & Tsoi, A.C. (2001). Noisy time series prediction using recurrent neural networks and grammatical inference. Machine learning 44, 1, 161-183.
$\\[0.05in]$

Giles, C.L., Sun, G.Z., Chen, H.H., Lee, Y.C. & Chen, D. (1989). Higher order recurrent networks and grammatical inference. NIPS, 380-387.
$\\[0.05in]$

Gisslen, L., Luciw, M., Graziano, V. & Schmidhuber, J. (2011). Sequential constant size compressor for reinforcement learning. In Proc. Fourth Conference on Artificial General Intelligence (AGI), Google, Mountain View, CA, 31–40. Springer.
$\\[0.05in]$

Gomez, F., Schmidhuber, J. & Miikkulainen, R. (2008). Accelerated neural evolution through cooperatively coevolved synapses. Journal of Machine Learning Research, 9, 937-965.
$\\[0.05in]$

Goodfellow, I., Bengio, Y. & Courville, A. (2016). Deep Learning, MIT Press.
$\\[0.05in]$

Graves, A. (2014). Generating sequences with recurrent neural networks. arXiv:1308.0850v5.
$\\[0.05in]$

Graves, A., Mohamed, A. & Hinton, G. (2013). Speech recognition with deep recurrent neural networks. arXiv:1303.5778.
$\\[0.05in]$

Graves, A. & Schmidhuber, J. (2005). Framewise phoneme classification with bidirectional LSTM and other neural network architectures. Neural Networks 18:5-6, 602-610.
$\\[0.05in]$

Greff, K., Srivastava, R.K., Koutnik, J., Steunebrink, B.R. & Schmidhuber, J. (2017). LSTM: A search space odyssey. IEEE transactions on neural
networks and learning systems, 28(10), 2222-2232.
$\\[0.05in]$

Gregor, K., Danihelka, I, Graves, A., Rezende, D.J. & Wierstra, D. (2015). DRAW: A recurrent neural network For image generation. In ICML, 2015.
$\\[0.05in]$

Grossberg, S. (1969). Some networks that can learn, remember, and reproduce any number of complicated space-time patterns, I. Journal of Mathematics and Mechanics, 19, 53–91.
$\\[0.05in]$

Grossberg, S. (1988). onlinear neural networks: Principles, mechanisms, and architectures. Neural Networks, 1, 17–61.
$\\[0.05in]$

Grossberg, S. (2013). Recurrent neural networks. Scholarpedia, 8(2):1888.
$\\[0.05in]$

Gu, S.H., Kelly, B.T. & Xiu, D.C (2020). Empirical asset pricing via machine learning. The Review of Financial Studies, Volume 33, Issue 5, 2223–2273.
$\\[0.05in]$

Gu, S.H., Kelly, B.T. & Xiu, D.C (2019). Autoencoder asset pricing models. Yale ICF Working Paper No. 2019-04; Chicago Booth Research Paper No. 19-24.
$\\[0.05in]$

Guo, T., Xu, Z., Yao, X, Chen, H.F., Aberer, K. & Funaya, K. (2016). Robust online time series prediction with recurrent neural networks. In Data
Science and Advanced Analytics (DSAA), 2016 IEEE International Conference, 816-825. Ieee, 2016.
$\\[0.05in]$

Hajizadeh, E., Seifi, A., Fazel Zarandi, M. & Turksen, I. (2012). A hybrid modeling approach for forecasting the volatility of S&P 500 index return. Expert Systems with Applications 39, 431–436.
$\\[0.05in]$

Hamid, S.A. & Iqbal, Z. (2004). Using neural networks for forecasting volatility of S&P 500 index futures prices. Journal of Business Research, 57, 1116-1125. 
$\\[0.05in]$

Harris, C.R., Millman, K.J., van der Walt, S.J., Gommers, R., Virtanen, P., Cournapeau, D., Wieser, E., Taylor, J., Berg, S., Smith, N.J., Kern, R., Picus, M., Hoyer, S., Brett, M., Haldane, A., Fernandez del Rio, A., Wiebe, M., Sheppard, K., Reddy, T., Weckesser, W. & Oliphant, T.E. (2020). Array programming with NumPy. Nature 585, 357–362.
$\\[0.05in]$

Hastie, T., Tibshirani, R. & Friedman, J. (2009). The elements of statistical learning: data mining, inference, and prediction. 2Ed, Springer.
$\\[0.05in]$

Heaton, J.B., Polson, N.G. & Witte, J.H. (2016). Deep learning in finance, arXiv:1602.06561.
$\\[0.05in]$

Hinton, G.E., Osindero, S. & Teh, Y.W. (2006). A fast learning algorithm for deep belief nets. Neural Computation, 18 (7).
$\\[0.05in]$

Hinton, G.E. & Salakhutdinov, R. (2006). Reducing the dimensionality of data with neural networks. Science, 313(5786), 504–507.
$\\[0.05in]$

Hinton, G.E., Srivastava, N., Krizhevsky, A., Sutskever, I. & Salakhutdinov, R.R. (2012). Improving neural networks by preventing co-adaptation
of feature detectors. arXiv:1207.0580.
$\\[0.05in]$

Hochreiter, S. (1991). Untersuchungen zu dynamischen neuronalen Netzen. Diploma thesis, Institut fur Informatik, Lehrstuhl Prof. Brauer, Technische Universitat Munchen.
$\\[0.05in]$

Hochreiter, S. & Schmidhuber, J. (1997). Long short-term memory. Neuralcomputation, 9(8): 1735-1780.
$\\[0.05in]$

Hochreiter, S., Bengio, Y., Paolo, F. & Schmidhuber, J. (2001). Gradient flow in recurrent nets: the difficulty of learning long-term dependencies. In Kremer & Kolen (Eds), A Field Guide to Dynamical Recurrent Neural Networks. IEEE Press.
$\\[0.05in]$

Hornik, K., Stinchcombe, M. & White, H. (1989). Multilayer feedforward networks are universal approximators. Neural networks 2, 359-366.
$\\[0.05in]$

Hornik, K. (1991). Approximation capabilities of multilayer feedforward networks. Neural Networks, 4(2), 251–257.
$\\[0.05in]$

Hsu, D. (2017). Time series forecasting based on augmented long short-term memory. arXiv:1707.00666.
$\\[0.05in]$

Hu, Y.M. & Tsoukalas, C. (1999). Combining conditional volatility forecasts using neural networks: an application to the EMS exchange rates. Journal of International Financial Markets, Institutions & Money 9, 407–422.
$\\[0.05in]$

Huck, N. (2009). Pairs selection and outranking: An application to the S&P 100 index. European Journal of Operational Research 196, 2, 819-825.
$\\[0.05in]$

Huck, N. (2010). Pairs trading and outranking: The multi-step-ahead forecasting case. European Journal of Operational Research 207, 3, 1702-1716.
$\\[0.05in]$

Huffman, D. A. (1952). A method for construction of minimum-redundancy codes. Proceedings IRE, 40, 1098–1101.
$\\[0.05in]$

Ioffe, S. & Szegedy, C. (2015). Batch normalization: accelerating deep network training by reducing internal covariate shift”. arXiv:1502.03167.
$\\[0.05in]$

Ivakhnenko, A. G. (1968). The group method of data handling – a rival of the method of stochastic approximation. Soviet Automatic Control, 13, 3, 43–55.
$\\[0.05in]$

Ivakhnenko, A. G. (1971). Polynomial theory of complex systems. IEEE Transactions on Systems, Man and Cybernetics, 4, 364–378.
$\\[0.05in]$

Ivakhnenko, A. G. & Lapa, V. G. (1965). Cybernetic Predicting Devices. CCM Information Corporation.
$\\[0.05in]$

Ivakhnenko, A. G., Lapa, V. G., & McDonough, R. N. (1967). Cybernetics and forecasting techniques.  American Elsevier, NY.
$\\[0.05in]$

Israel, R., Kelly, B.T. & Moskowitz, T. (2020). Can machines "learn" finance? Journal of Investment Management, Volume 18, No. 2.
$\\[0.05in]$

Johnson, M.J., Duvenaud, D., Wiltschko, A.B., Datta, S.R. & Adams, R.P. (2016). Composing graphical models with neural networks for structured representations and fast inference. In NIPS, 2016.
$\\[0.05in]$

Kamijo, K. & Tanigawa, T. (1990). Stock price pattern recognition - a recurrent neural network approach. 1990 IJCNN International Joint Conference on Neural Networks.
$\\[0.05in]$

Karl, M., Soelch, M., Bayer, J. & van der Smagt, P. (2017). Deep variational Bayes filters: Unsupervised learning of state space models from raw data. In ICLR, 2017.
$\\[0.05in]$

Karpathy, A., Johnson, J. & Li, F.F. (2015). Visualizing and understanding recurrent networks. arXiv:1506.02078.
$\\[0.05in]$

Khan, A.I. (2011). Financial volatility forecasting by nonlinear support vector machine heterogeneous autoregressive model: Evidence from Nikkei 225 Stock Index. International Journal of Economics and Finance 3, 138–150.
$\\[0.05in]$

Kingma, D.P. & Welling, M. (2014). Auto-Encoding Variational Bayes. In ICLR, 2014.
$\\[0.05in]$

Kelley, H. J. (1960). Gradient theory of optimal flight paths. ARS Journal, 30, 10, 947–954.
$\\[0.05in]$

Kim, T. & Kim, H.Y. (2019). Forecasting stock prices with a feature fusion LSTM-CNN model using different representations of the same data. PLoS ONE 14(2): e0212320.
$\\[0.05in]$

Kohonen, T. (1988). Self-Organization and Associative Memory. Springer, second edition.
$\\[0.05in]$

kornblith, S., Norouzi, M., Lee, H. & Hinton, G.E. (2019). Similarity of neural network representations revisited. Proceedings of the 36th International Conference on Machine Learning, Long Beach, California.
$\\[0.05in]$

Laptev, N., Yosinski, J., Li, L.E. & Smyl, S. (2017). Time-series extreme event forecasting with neural networks at Uber. In International
Conference on Machine Learning, 34, pages 1-5.
$\\[0.05in]$

LeCun, Y. (1985). Une proc´edure d’apprentissage pour r´eseau `a seuil asym´etrique. Proceedings of Cognitiva 85, Paris, 599–604.
$\\[0.05in]$

LeCun, Y. (1988). A theoretical framework for back-propagation. In Touretzky, D., Hinton, G., & Sejnowski, T., editors, Proceedings of the 1988 Connectionist Models Summer School, 21–28, CMU, Pittsburgh, Pa. Morgan Kaufmann.
$\\[0.05in]$

LeCun, Y., Bengio, Y. & Hinton, G. (2015). Deep learning. Nature, 521(7553): 436-444.
$\\[0.05in]$

Lipton, Z., Berkowitz, J. & Elkan, C. (2015). A critical review of recurrent neural networks for sequence learning. arXiv:1506.00019.
$\\[0.05in]$

Liu, F., Pantelous, A.A. & von Mettenheim, H.J. (2018). Forecasting and trading high frequency volatility on large indices. Quantitative Finance, Vol. 18, 5, 737-748
$\\[0.05in]$

Luo, R., Zhang, W.N., Xu, X.J. & Wang, J. (2018). A neural stochastic volatility model. The Thirty-Second AAAI Conference on Artificial Intelligence (AAAI-18), 6401-6408.
$\\[0.05in]$

Malhotra, P., Vig, L, Shroff, G. & Agarwal, P. (2015). Long short term memory networks for anomaly detection in time series. In Proceedings, 89. Presses universitaires de Louvain, 2015.
$\\[0.05in]$

Malliaris, M. & Salchenberger, L. (1996). Using neural networks to forecast the S&P 100 implied volatility. Neurocomputing, 10(2), 183–195.
$\\[0.05in]$

Mayer, H., Gomez, F., Wierstra, D., Nagy, I., Knoll, A. & Schmidhuber, J. (2008). A system for robotic heart surgery that learns to tie knots using recurrent neural networks. Advanced Robotics, 22/13-14, 1521-1537.
$\\[0.05in]$

Minsky, M.L. & Papert, S.A. (1990). Perceptrons, 2nd Ed. MIT Press, Cambridge MA.
$\\[0.05in]$

Nguyen, N., Tran, M.N., Gunawan, D. & Kohn, R. (2019). A long short-term memory stochastic volatility model. arXiv:1906.02884v2 [
econ.EM] 30 Sep 2019.
$\\[0.05in]$

Perez-Ortiz, J. A., Gers, F. A., Eck, D., & Schmidhuber, J. (2003). Kalman filters improve LSTM network performance in problems unsolvable by traditional recurrent nets. Neural Networks, 16, 241–250.
$\\[0.05in]$

Pachitariu, M. & Sahani, M. (2012). Learning visual motion in recurrent neural networks. In NIPS, 2012.
$\\[0.05in]$

Pascanu, R., Mikolov, T. & Bengio, Y. (2013). On the difficulty of training recurrent neural networks. In International Conference on Machine Learning, 1310–1318.
$\\[0.05in]$

Patel, A.B., Nguyen, T. & Baraniuk, R.G. (2015). A probabilistic theory of deep learning. Rice University Electrical and Computer Engineering Dept. Technical Report No 2015-1. arXiv:1504.00641
$\\[0.05in]$

Petnehazi, G. & Gall, J. (2019). Exploring the predictability of range‐based volatility estimators using recurrent neural networks. Intell Sys Acc Fin Mgmt. 2019; 1–8.
$\\[0.05in]$

Petnehazi, G. (2019). Recurrent neural networks for time series forecasting. arXiv:1901.00069v1 [cs.LG].
$\\[0.05in]$

Rezende, D.J., Mohamed, S. & Wierstra, D. (2014). Stochastic backpropagation and approximate inference in deep generative models. In ICML, 2014.
$\\[0.05in]$

Richard, J.P. (2003). Time delay systems: An overview of some recent advances and open problems. Automatica, 39, 10, 1667–1694.
$\\[0.05in]$

Russell, S. J., Norvig, P., Canny, J. F., Malik, J. M., & Edwards, D. D. (1995). Artificial Intelligence: a Modern Approach, Vol 2. Englewood Cliffs: Prentice Hall.
$\\[0.05in]$

Schmidhuber, J. (1992). Learning complex, extended sequences using the principle of history compression. Neural Computation, 4(2), 234–242.
$\\[0.05in]$

Schmidhuber, J., Wierstra, D., Gagliolo, M. & Gomez, F. (2007). Training recurrent networks by Evolino. Neural Computation, 19(3), 757-779.
$\\[0.05in]$

Schmidhuber, J. (2013). My first Deep Learning system of 1991 + Deep Learning timeline 1962-2013. Technical Report arXiv:1312.5548v1 [cs.NE], The Swiss AI Lab IDSIA.
$\\[0.05in]$

Schmidhuber, J. (2015). Deep learning in neural networks: An overview. Neural networks, 61, 85–117.
$\\[0.05in]$

Sezer, O.B., Gudelek, M.U. & Ozbayoglu, A.M. (2019). Financial time series forecasting with deep learning: a systematic literature review 2005-2019. arXiv:1911.13288v1
$\\[0.05in]$

Sherstinsky, A. & Picard, R.W. (1998). On stability and equilibria of the M-Lattice. IEEE Transactions on Circuits and Systems – I: Fundamental Theory and Applications, 45, 4, 408–415.
$\\[0.05in]$

Sherstinsky, A. (2018a). Deriving the Recurrent Neural Network Definition and RNN Unrolling Using Signal Processing. In Critiquing and Correcting Trends in Machine Learning Workshop, NeurIPS 31.
2018), Dec 2018.
$\\[0.05in]$

Sherstinsky, A. (2018b). Fundamentals of Recurrent Neural Network (RNN) and Long Short-Term Memory (LSTM) Network, arxiv:1808.03314.
$\\[0.05in]$

Sherstinsky, A. (2020). Fundamentals of recurrent neural network (RNN) and long short-term memory (LSTM) network. Physica D: Nonlinear Phenomena, Volume 404, March 2020: Special Issue on Machine Learning and Dynamical Systems.
$\\[0.05in]$

Siegelmann, H. T. & Sontag, E. D. (1991). Turing computability with neural nets. Applied Mathematics Letters, 4(6), 77–80.
$\\[0.05in]$

Soltani, R. & Jiang, H. (2016). Higher order recurrent neural networks. arXiv:1605.00064.
$\\[0.05in]$

Srivastava, N., Hinton, G., Krizhevsky, A., Sutskever, I. & Salakhutdinov, R. (2014). Dropout: A simple way to prevent neural networks from over-fitting. Journal of Machine Learning Research, 15(1), 1929-1958.
$\\[0.05in]$

Sutskever, I., Martens, J. & Hinton, G.E. (2011). Generating text with recurrent neural networks. In Proceedings of the 28th International Conference on Machine Learning (ICML-11), 1017-1024.
$\\[0.05in]$

Tang, Z.Y., De Almeida, C. & Fishwick, P.A. (1991). Time series forecasting using neural networks vs. box-jenkins methodology. Simulation, 57(5), 303-310, 1991.
$\\[0.05in]$

Takeuchi, L. & Lee, Y.Y. (2013). Applying deep learning to enhance momentum trading strategies in stocks. Working paper, Stanford University.
$\\[0.05in]$

Verstyuk, S. (2019). Modeling multivariate time series in economics: from auto-regressions to recurrent neural networks. Unpublished paper.
$\\[0.05in]$

Van der Westhuizen, J. &  Lasenby, J. (2017). Visualizing LSTM decisions. Stat, 1050:23.
$\\[0.05in]$

White, H. (1988). Economic prediction using neural networks: the case of IBM daily stock returns. IEEE 1988 International Conference on Neural Networks.
$\\[0.05in]$

Xiong, R.X., Nichols, E.P. & Shen, Y. (2016). Deep learning stock volatility with Google domestic trends. arXiv:1512.04916v3.
$\\[0.05in]$

Yu, R., Zheng, S. Anandkumar, A. & Yue, Y. (2019). Long-term forecasting using higher-order tensor RNNs. Journal of Machine Learning Research 1 (2019) 1-48.
$\\[0.05in]$

Zaheer, M., Ahmed, A. & Smola, A.J. (2017). Latent LSTM allocation: Joint clustering and non-linear dynamic modeling of sequence data. International Conference on Machine Learning, Sydney, Australia, PMLR 70.
$\\[0.05in]$

Zhang, P.G. & Min, Q. (2005). Neural network forecasting for seasonal and trend time series. European journal of operational research, 160(2), 501-514.
$\\[0.05in]$

Zheng, S., Yue, Y.S., & Lucey, P. (2016). Generating long-term trajectories using deep hierarchical networks. In Advances in Neural Information Processing Systems, 1543-1551.
$\\[0.05in]$

Zhu, L.X. & Laptev, N. (2017). Deep and confident prediction for time series at Uber. In Data Mining Workshops (ICDMW), 2017 IEEE International
Conference, 103-110. IEEE, 2017.
$\\[0.05in]$